In [3]:
!pip install dune_client

     -------------------------------------- 58.7/58.7 kB 768.7 kB/s eta 0:00:00


In [6]:
!pip install dataclasses_json


     -------------------------------------- 49.4/49.4 kB 832.4 kB/s eta 0:00:00


In [3]:
import streamlit as st
import pandas as pd
import requests
import numpy as np
import yfinance as yf
from dune_client.client import DuneClient

In [4]:
@st.cache_data()
def fetch_data_from_api(api_url, params=None):
    response = requests.get(api_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'rows' in data['result']:
            return pd.DataFrame(data['result']['rows'])
        return data
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return pd.DataFrame()  # or an empty dict

2024-04-03 16:31:27.182 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [5]:
def fetch_historical_data(api_url, api_key):
    # Use the API key either as a query parameter or in the headers
    params = {'vs_currency': 'usd', 'days': 'max', 'interval': 'daily', 'x_cg_demo_api_key': api_key}
    headers = {'x-cg-demo-api-key': api_key}  # Alternatively, use this header

    response = requests.get(api_url, params=params, headers=headers)

    if response.status_code == 200:
        # Parse the JSON response
        historical_pricedata = response.json()
        # Extract the 'prices' and 'market_caps' data
        historical_price = historical_pricedata['prices']
        market_cap = pd.DataFrame(historical_pricedata['market_caps'], columns=['date', 'marketcap'])

        # Convert the 'timestamp' column from UNIX timestamps in milliseconds to datetime objects
        history = pd.DataFrame(historical_price, columns=['timestamp', 'price'])
        history['date'] = pd.to_datetime(history['timestamp'], unit='ms')
        history.set_index('date', inplace=True)
        history.drop(columns='timestamp', inplace=True)

        vol = pd.DataFrame(historical_pricedata['total_volumes'], columns=['date', 'volume'])
        vol['date'] = pd.to_datetime(vol['date'], unit='ms')
        vol.set_index('date', inplace=True)
        
        return history, market_cap, vol
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

In [6]:
api_key_dune = st.secrets["api_key"]
api_key_cg = st.secrets["api_key_cg"]
api_key_FRED = st.secrets["FRED_API_KEY"]

In [7]:
dune = DuneClient(api_key_dune)

In [8]:
def fetch_dune_data(num):
    result = dune.get_latest_result(num)
    return pd.DataFrame(result.result.rows)

In [9]:
pd.options.display.float_format = '{:,.2f}'.format

### First, lets get MakerDAO Financial Statements from https://dune.com/steakhouse/makerdao

Balance Sheet

In [10]:
# Balance Sheet
#bs_raw = dune.get_latest_result(2840463)

In [11]:
#bs_df = pd.DataFrame(bs_raw.result.rows)
#bs_df['period'] = pd.to_datetime(bs_df['period'])
#bs_df.set_index('period', inplace=True)
#bs_df.index = bs_df.index.normalize()
#bs_df = bs_df.sort_index()

In [12]:
#bs_df

In [13]:
bs_path = '../data/csv/bs.csv'

In [14]:
#bs_df.to_csv(bs_path)

In [15]:
bs_csv = pd.read_csv(bs_path, index_col='period', parse_dates=True)

In [16]:
#categorizing items as asset, liability, or equity
def categorize_item(item):
    if item in ['Crypto-Loans', 'Real-World Assets', 'Others assets', 'Stablecoins']:
        return 'Assets'
    elif item in ['DAI','DSR']:  # Assuming DAI represents a liability here; adjust according to your accounting rules
        return 'Liabilities'
    elif item == 'Equity':
        return 'Equity'
    else:
        return 'Other'  # For any item not explicitly categorized

# Assuming 'df' is your DataFrame
bs_csv['category'] = bs_csv['item'].apply(categorize_item)



In [17]:
bs_csv = bs_csv.iloc[::-1]

In [441]:
bs_csv.describe()

,balance,normalized
count,"8,549.00","8,549.00"
mean,0.00,0.00
std,"2,676,252,421.26",0.47
min,"-9,951,190,219.65",-0.99
25%,"-81,183,521.56",-0.01
50%,"46,794.81",0.00
75%,"1,333,547,688.40",0.30
max,"6,135,551,191.72",1.00


In [19]:
pivoted_balance_sheet = bs_csv.pivot(columns='item', values='balance')

In [20]:
# Percent Changes in account balances 
pivoted_balance_sheet['Crypto-Loans_pct_chg'] = pivoted_balance_sheet['Crypto-Loans'].pct_change()
pivoted_balance_sheet['DAI_pct_chg'] = pivoted_balance_sheet['DAI'].pct_change()
pivoted_balance_sheet['DSR_pct_chg'] = pivoted_balance_sheet['DSR'].pct_change()
pivoted_balance_sheet['Equity_pct_chg'] = pivoted_balance_sheet['Equity'].pct_change()
pivoted_balance_sheet['Others_assets_pct_chg'] = pivoted_balance_sheet['Others assets'].pct_change()  # Assuming this is the correct column name
pivoted_balance_sheet['Real-World_Assets_pct_chg'] = pivoted_balance_sheet['Real-World Assets'].pct_change()
pivoted_balance_sheet['Stablecoins_pct_chg'] = pivoted_balance_sheet['Stablecoins'].pct_change()


In [21]:
# Rolling Averages, Standard Deviation
# Define the window size for rolling calculation
window_size = 7  # for example, a 7-day rolling window

# Calculate rolling averages
pivoted_balance_sheet['Crypto-Loans_7d_rolling_avg'] = pivoted_balance_sheet['Crypto-Loans'].rolling(window=window_size).mean()
pivoted_balance_sheet['DAI_7d_rolling_avg'] = pivoted_balance_sheet['DAI'].rolling(window=window_size).mean()
pivoted_balance_sheet['DSR_7d_rolling_avg'] = pivoted_balance_sheet['DSR'].rolling(window=window_size).mean()
pivoted_balance_sheet['Equity_7d_rolling_avg'] = pivoted_balance_sheet['Equity'].rolling(window=window_size).mean()
pivoted_balance_sheet['Others_assets_7d_rolling_avg'] = pivoted_balance_sheet['Others assets'].rolling(window=window_size).mean()
pivoted_balance_sheet['Real-World_Assets_7d_rolling_avg'] = pivoted_balance_sheet['Real-World Assets'].rolling(window=window_size).mean()
pivoted_balance_sheet['Stablecoins_7d_rolling_avg'] = pivoted_balance_sheet['Stablecoins'].rolling(window=window_size).mean()

# Calculate volatility (standard deviation)
pivoted_balance_sheet['Crypto-Loans_7d_rolling_volatility'] = pivoted_balance_sheet['Crypto-Loans_pct_chg'].rolling(window=window_size).std()
pivoted_balance_sheet['DAI_7d_rolling_volatility'] = pivoted_balance_sheet['DAI_pct_chg'].rolling(window=window_size).std()
pivoted_balance_sheet['DSR_7d_rolling_volatility'] = pivoted_balance_sheet['DSR_pct_chg'].rolling(window=window_size).std()
pivoted_balance_sheet['Equity_7d_rolling_volatility'] = pivoted_balance_sheet['Equity_pct_chg'].rolling(window=window_size).std()
pivoted_balance_sheet['Others_assets_7d_rolling_volatility'] = pivoted_balance_sheet['Others_assets_pct_chg'].rolling(window=window_size).std()
pivoted_balance_sheet['Real-World_Assets_7d_rolling_volatility'] = pivoted_balance_sheet['Real-World_Assets_pct_chg'].rolling(window=window_size).std()
pivoted_balance_sheet['Stablecoins_7d_rolling_volatility'] = pivoted_balance_sheet['Stablecoins_pct_chg'].rolling(window=window_size).std()



In [22]:
# Assuming you've already calculated percent changes (_pct_chg)
window_size = 30  # Adjust based on your analysis needs

# Calculate rolling averages and volatilities based on percent changes
for col in ['Crypto-Loans', 'DAI', 'DSR', 'Equity', 'Others_assets', 'Real-World_Assets', 'Stablecoins']:
    pct_chg_col = f'{col}_pct_chg'  # The column names for percent changes you've calculated
    pivoted_balance_sheet[f'{col}_30d_rolling_avg_pct_chg'] = pivoted_balance_sheet[pct_chg_col].rolling(window=window_size).mean()
    pivoted_balance_sheet[f'{col}_30d_volatility_pct_chg'] = pivoted_balance_sheet[pct_chg_col].rolling(window=window_size).std()


In [23]:
pivoted_balance_sheet.columns = [f'b_s_{col}' if col != 'period' else col for col in pivoted_balance_sheet.columns]

In [24]:
pivoted_balance_sheet.tail()

,b_s_Crypto-Loans,b_s_DAI,b_s_DSR,b_s_Equity,b_s_Others assets,b_s_Real-World Assets,b_s_Stablecoins,b_s_Crypto-Loans_pct_chg,b_s_DAI_pct_chg,b_s_DSR_pct_chg,...,b_s_DSR_rolling_avg_pct_chg,b_s_DSR_volatility_pct_chg,b_s_Equity_rolling_avg_pct_chg,b_s_Equity_volatility_pct_chg,b_s_Others_assets_rolling_avg_pct_chg,b_s_Others_assets_volatility_pct_chg,b_s_Real-World_Assets_rolling_avg_pct_chg,b_s_Real-World_Assets_volatility_pct_chg,b_s_Stablecoins_rolling_avg_pct_chg,b_s_Stablecoins_volatility_pct_chg
period,,,,,,,,,,,,,,,,,,,,,
2024-03-17 00:00:00+00:00,"2,567,732,340.42","-3,225,717,987.93","-1,405,341,920.91","-68,936,169.75","48,533.09","1,082,824,203.90","1,049,391,001.19",-0.02,-0.01,0.03,...,0.01,0.05,0.01,0.04,0.00,0.00,-0.02,0.03,0.03,0.17
2024-03-18 00:00:00+00:00,"2,487,739,660.28","-3,236,587,097.37","-1,440,743,920.84","-67,073,216.58","48,533.09","1,084,684,721.77","1,171,931,319.66",-0.03,0.00,0.03,...,0.01,0.05,0.01,0.04,0.00,0.00,-0.02,0.03,0.03,0.17
2024-03-19 00:00:00+00:00,"2,442,828,415.48","-3,237,504,446.80","-1,399,485,314.96","-66,776,806.16","48,533.09","1,084,527,447.54","1,176,362,171.82",-0.02,0.00,-0.03,...,0.01,0.05,0.01,0.04,0.00,0.00,-0.02,0.03,0.03,0.17
2024-03-20 00:00:00+00:00,"2,358,443,170.28","-3,225,358,948.44","-1,373,045,029.84","-67,160,889.57","48,533.09","1,083,706,110.61","1,223,367,053.87",-0.03,-0.00,-0.02,...,0.01,0.05,0.01,0.04,0.00,0.00,-0.02,0.03,0.03,0.17
2024-03-21 00:00:00+00:00,"2,358,355,394.97","-3,223,114,058.66","-1,384,700,394.26","-67,156,270.08","48,533.09","1,083,706,110.61","1,232,860,684.32",-0.00,-0.00,0.01,...,0.01,0.05,0.01,0.04,0.00,0.00,-0.02,0.03,0.03,0.17


In [25]:
pivoted_balance_sheet.shape[0]

1360

MONTHLY Income Statement/PnL (also includes more detailed balance sheet)

In [26]:
#is_df = fetch_dune_data(2641549) 

In [27]:
#is_df.head()

In [28]:
#is_df_wide = is_df.pivot_table(index='period', columns='item', values='value', aggfunc='sum').reset_index()
#is_df_wide = is_df_wide.iloc[::-1]

In [29]:
#is_df_wide.head()

In [30]:
is_path = '../data/csv/is.csv'

In [31]:
#is_df.to_csv(is_path)

In [32]:
is_csv = pd.read_csv(is_path, index_col='period', parse_dates=True)

In [386]:
cleaned_is = is_csv.drop(columns=['Unnamed: 0','year'])

In [442]:
cleaned_is.describe()

,expenses,lending_income,liquidation_income,month,net_income,trading_income,value
count,106.00,106.00,106.00,954.00,106.00,106.00,689.00
mean,"-2,138,542.90","4,914,952.32","851,312.42",6.43,"3,835,126.96","207,405.12","1,346,134,004.51"
std,"2,317,856.06","5,635,119.85","3,083,911.34",3.60,"6,165,521.06","387,439.00","2,368,584,436.46"
min,"-13,632,119.60","2,828.77","-5,787,399.49",1.00,"-5,601,718.87",0.00,"-13,632,119.60"
25%,"-2,990,860.00","993,839.65","4,623.88",3.00,"36,867.28",0.00,0.00
50%,"-1,820,097.71","2,965,648.32","42,700.73",6.00,"2,136,663.16","9,359.24","3,044,390.27"
75%,0.00,"7,786,060.64","205,622.81",10.00,"6,547,116.47","229,736.22","2,038,968,283.83"
max,0.00,"25,123,953.08","18,025,406.62",12.00,"23,542,837.91","1,556,244.79","9,429,355,721.67"


In [34]:
pivoted_income_statement = is_csv.pivot_table(index='period', 
                            columns='item', 
                            values='value', 
                            aggfunc='sum').reset_index()

In [35]:
pivoted_income_statement['Total Revenues']= pivoted_income_statement[['1 - PnL', '1.1 - Lending Revenues', '1.2 - Liquidations Revenues', '1.3 - Trading Revenues']].sum(axis=1)
pivoted_income_statement['Total Expenses'] = pivoted_income_statement[['1.4 - Lending Expenses', '1.5 - Liquidations Expenses', '1.6 - Workforce Expenses']].sum(axis=1)
pivoted_income_statement['profit_margin'] = pivoted_income_statement['1.9 - Net Income'] / pivoted_income_statement['2.9 - Total Assets']
pivoted_income_statement['ROA'] = pivoted_income_statement['1.9 - Net Income'] / pivoted_income_statement['2.9 - Total Assets']
pivoted_income_statement['ROE'] = pivoted_income_statement['1.9 - Net Income'] / pivoted_income_statement['3.7 - Equity (Surplus Buffer)']
pivoted_income_statement['debt_to_equity'] = pivoted_income_statement['3.1 - Liabilities (DAI)'] / pivoted_income_statement['3.7 - Equity (Surplus Buffer)']
pivoted_income_statement['debt_ratio'] = pivoted_income_statement['3.1 - Liabilities (DAI)'] / pivoted_income_statement['2.9 - Total Assets'] 
pivoted_income_statement['cumulative_revenues'] = pivoted_income_statement['Total Revenues'].cumsum()
pivoted_income_statement['cumulative_expenses'] = pivoted_income_statement['Total Expenses'].cumsum()
pivoted_income_statement['cumulative_net_income'] = pivoted_income_statement['1.9 - Net Income'].cumsum()


In [36]:
pivoted_income_statement.tail()

item,period,1 - PnL,1.1 - Lending Revenues,1.2 - Liquidations Revenues,1.3 - Trading Revenues,1.4 - Lending Expenses,1.5 - Liquidations Expenses,1.6 - Workforce Expenses,1.9 - Net Income,2 - Assets,...,Total Revenues,Total Expenses,profit_margin,ROA,ROE,debt_to_equity,debt_ratio,cumulative_revenues,cumulative_expenses,cumulative_net_income
48,2023-11-01,0.00,"29,666,040.79",455.03,0.00,"-6,376,789.60",0.00,"-2,974,302.25","20,315,403.96",0.00,...,"29,666,495.81","-9,351,091.85",0.00,0.00,0.35,90.89,0.99,"291,977,550.62","-133,751,913.34","158,225,637.28"
49,2023-12-01,0.00,"14,270,261.48",0.00,0.00,"-6,484,200.84",0.00,"-2,658,930.56","5,127,130.08",0.00,...,"14,270,261.48","-9,143,131.40",0.00,0.00,0.10,98.65,0.99,"306,247,812.10","-142,895,044.74","163,352,767.36"
50,2024-01-01,0.00,"20,751,255.62","27,376.30",0.00,"-5,943,924.23",0.00,"-5,669,555.33","9,165,152.35",0.00,...,"20,778,631.92","-11,613,479.57",0.00,0.00,0.17,91.96,0.99,"327,026,444.02","-154,508,524.31","172,517,919.71"
51,2024-02-01,0.00,"29,691,662.12",0.00,0.00,"-4,567,709.04",0.00,"-1,581,115.17","23,542,837.91",0.00,...,"29,691,662.12","-6,148,824.21",0.00,0.00,0.35,74.02,0.99,"356,718,106.13","-160,657,348.51","196,060,757.62"
52,2024-03-01,0.00,"18,130,771.84","4,623.88",0.00,"-7,076,544.94",0.00,"-3,857,879.46","7,200,971.32",0.00,...,"18,135,395.72","-10,934,424.40",0.00,0.00,0.11,68.66,0.99,"374,853,501.85","-171,591,772.91","203,261,728.94"


In [37]:
# Percent Changes in account balances
window_size = 3  # Three months
# Calculate rolling averages and volatilities based on percent changes
for col in ['Total Revenues', 'Total Expenses', '1.9 - Net Income']:
    pct_chg_col = f'{col}_pct_chg'  # Define the percent change column name
    pivoted_income_statement[pct_chg_col] = pivoted_income_statement[col].pct_change()
    # Use the pct_chg_col variable correctly now
    pivoted_income_statement[f'{col}_3m_rolling_avg_pct_chg'] = pivoted_income_statement[pct_chg_col].rolling(window=window_size).mean()
    pivoted_income_statement[f'{col}_3m_volatility_pct_chg'] = pivoted_income_statement[pct_chg_col].rolling(window=window_size).std()
    for lag in range(1,13):
        pivoted_income_statement[f'{col}_3m_rolling_avg_pct_chg_lag_{lag}'] = pivoted_income_statement[f'{col}_3m_rolling_avg_pct_chg'].shift(lag)
        pivoted_income_statement[f'{col}_3m_volatility_pct_chg_lag_{lag}'] = pivoted_income_statement[f'{col}_3m_volatility_pct_chg'].shift(lag)


In [38]:

# Generate lagged features
for lag in range(1, 13):  # From 1 to 12 months
    pivoted_income_statement[f'Total_Revenues_Lag_{lag}m'] = pivoted_income_statement['Total Revenues'].shift(lag)
    pivoted_income_statement[f'Total_Expenses_Lag_{lag}m'] = pivoted_income_statement['Total Expenses'].shift(lag)
    pivoted_income_statement[f'Net_Income_Lag_{lag}m'] = pivoted_income_statement['1.9 - Net Income'].shift(lag)
    pivoted_income_statement[f'profit_margin_Lag_{lag}m'] = pivoted_income_statement['profit_margin'].shift(lag)
    pivoted_income_statement[f'ROA_Lag_{lag}m'] = pivoted_income_statement['ROA'].shift(lag)
    pivoted_income_statement[f'ROE_Lag_{lag}m'] = pivoted_income_statement['ROE'].shift(lag)
    pivoted_income_statement[f'debt_to_equity_Lag_{lag}m'] = pivoted_income_statement['debt_to_equity'].shift(lag)
    pivoted_income_statement[f'debt_ratio_Lag_{lag}m'] = pivoted_income_statement['debt_ratio'].shift(lag)


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_10768\259549745.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pivoted_income_statement[f'debt_ratio_Lag_{lag}'] = pivoted_income_statement['debt_ratio'].shift(lag)
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_10768\259549745.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pivoted_income_statement[f'Total_Revenues_Lag_{lag}'] = pivoted_income_statement['Total Revenues'].shift(lag)
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_10768\259549745.py:4: PerformanceWa

In [39]:
pivoted_income_statement.shape[0]

53

Assets/Revenue Per Type
Coinbase asset type: http://forum.makerdao.com/t/mip81-coinbase-usdc-institutional-rewards/17703/254?u=sebventures


In [40]:
#assets_raw = dune.get_latest_result(58495)

In [41]:
#assets_p_t_ts = pd.DataFrame(assets_raw.result.rows)
#assets_p_t_ts['dt'] = pd.to_datetime(assets_p_t_ts['dt'])
#assets_p_t_ts.set_index('dt', inplace=True)

In [42]:
#assets_p_t_ts.head()

In [43]:
#assets_p_t_ts.to_csv(as_path)

In [44]:
as_path = '../data/csv/as.csv'

In [45]:
as_csv = pd.read_csv(as_path, index_col='dt', parse_dates=True)

In [46]:
as_csv = as_csv.drop(columns=['total_asset'])

Daily Interest Revenues By Vault

In [47]:
#ir_v = fetch_dune_data(3567939) 

In [48]:
#ir_v['period'] = pd.to_datetime(ir_v['period'])
#ir_v.set_index('period', inplace=True)

In [49]:
#ir_v.head()

In [50]:
daily_int_path = '../data/csv/d_int.csv'

In [51]:
#ir_v.to_csv(daily_int_path)

In [52]:
ir_csv = pd.read_csv(daily_int_path, index_col='period', parse_dates=True)

In [53]:
ir_csv.shape[0]

22031

In [54]:

ir_csv = ir_csv.rename_axis('day')

 

In [55]:
ir_csv = ir_csv.rename(columns={'collateral':'ilk'})

In [56]:
ir_csv['ilk'].unique()

array(['ETH-A', 'ETH-B', 'ETH-C', 'WBTC-A', 'WBTC-B', 'WBTC-C',
       'WSTETH-A', 'WSTETH-B', 'RWA002-A', 'RWA013-A', 'DIRECT-SPARK-DAI',
       'RWA014-A', 'RWA005-A', 'RWA012-A', 'RWA015-A', 'RWA007-A',
       'RETH-A', 'RWA003-A', 'GUNIV3DAIUSDC2-A', 'CRVV1ETHSTETH-A',
       'USDC-B', 'LINK-A', 'MATIC-A', 'UNIV2USDCETH-A', 'GNO-A',
       'UNIV2DAIUSDC-A', 'YFI-A', 'RWA004-A', 'GUNIV3DAIUSDC1-A',
       'GUSD-A', 'PAXUSD-A', 'USDC-A', 'DIRECT-AAVEV2-DAI',
       'DIRECT-COMPV2-DAI', 'RWA008-A', 'RENBTC-A', 'MANA-A', 'RWA009-A',
       'RWA001-A', 'UNI-A', 'UNIV2DAIETH-A', 'UNIV2WBTCETH-A',
       'UNIV2WBTCDAI-A', 'RWA-001', 'UNIV2UNIETH-A', 'TUSD-A', 'USDP-A',
       'BAT-A', 'BAL-A', 'ZRX-A', 'COMP-A', 'PSM-GUSD-A', 'AAVE-A',
       'UNIV2LINKETH-A', 'KNC-A', 'LRC-A', 'PSM-USDC-A', 'UNIV2AAVEETH-A',
       'UNIV2DAIUSDT-A', 'UNIV2ETHUSDT-A', 'USDT-A', 'PSM-PAX-A',
       'RWA006-A', 'PAX-A', nan, 'USDC', 'SAI'], dtype=object)

In [57]:
top_vaults = ir_csv.groupby('ilk').sum().sort_values('daily_revenues', ascending=False)

In [58]:
#10 Most Revenue Generating Vaults
top_10_vaults = top_vaults.head(10)

In [59]:
top_10_vaults

,daily_revenues
ilk,
ETH-A,"110,769,338.09"
WBTC-A,"41,088,135.60"
WSTETH-A,"22,414,721.27"
ETH-C,"17,297,706.33"
ETH-B,"15,374,386.83"
WSTETH-B,"11,554,717.49"
USDC-A,"7,351,192.96"
RWA013-A,"3,241,981.74"
WBTC-C,"2,697,915.32"


Dai Maturity Profile

Step-by-Step Process:
Tracking DAI Movements:

First, all transactions involving DAI are tracked to understand how DAI moves in and out of wallets. This includes both inflows (adding DAI to a wallet) and outflows (removing DAI from a wallet).
Defining Maturity Buckets:

Maturity buckets are predefined categories based on time durations, such as "1-day", "1-week", "1-month", "1-year", etc. Each bucket represents a hypothesis about how long DAI tends to stay put before being moved again.
Assigning Weights to Buckets:

Weights are assigned to each maturity bucket to reflect assumptions or historical observations about the distribution of DAI across these buckets. For example, if historically 30% of DAI is moved or used within a day, then the "1-day" bucket might get a weight of 0.30 (or 30%).
Applying Weights Based on Wallet Types:

DAI can be held in different types of wallets or contracts, each with its own expected behavior. For example, DAI in a savings contract (like DSR) might be considered more long-term ("1-year"), while DAI in a regular wallet might be more liquid ("1-day" or "1-week"). The weights applied to the DAI in these wallets reflect these expectations.
Calculating DAI Amounts per Bucket:

For each wallet (or DAI holding), the total amount of DAI is distributed across the maturity buckets based on the assigned weights. This means if a wallet has 100 DAI and the "1-day" bucket weight is 30%, then 30 DAI is considered to have a 1-day maturity.
The process is repeated for each wallet and each maturity bucket, based on the specific weights for that wallet type and the total DAI it holds.
Aggregating Across the Ecosystem:

Finally, to get the ecosystem-wide view, the amounts of DAI in each maturity bucket from all wallets are aggregated. This provides a snapshot of how much DAI is considered to be in each maturity bucket across the entire MakerDAO system at any given time.

In [60]:
#d_m = fetch_dune_data(907852)

In [61]:
#d_m['dt'] = pd.to_datetime(d_m['dt'])

In [62]:
#d_m

In [63]:

#d_m.to_csv(d_m_path)


In [396]:
d_m_path = '../data/csv/d_m.csv'

In [397]:
d_m_csv = pd.read_csv(d_m_path, index_col='dt', parse_dates=True)

In [400]:
print(d_m_csv.describe())

       Unnamed: 0          outflow  outflow_dai_only  outflow_surplus_buffer  \
count    9,528.00         9,528.00          9,528.00                9,456.00   
mean     4,763.50   730,298,227.76    723,044,786.34            7,308,670.67   
std      2,750.64 1,172,359,348.27  1,153,238,025.26           20,455,276.75   
min          0.00             0.00              0.00           -5,674,219.65   
25%      2,381.75   107,656,317.16    107,636,256.59                    0.00   
50%      4,763.50   323,980,629.04    323,980,629.04                    0.00   
75%      7,145.25   628,916,174.70    628,916,174.70                    0.00   
max      9,527.00 6,014,959,678.79  5,952,992,110.30           83,550,327.25   

           total_period  
count          9,528.00  
mean   4,381,789,366.59  
std    2,882,217,947.64  
min                0.00  
25%    1,093,258,152.24  
50%    5,189,669,321.25  
75%    6,203,283,988.26  
max   10,118,832,636.48  


In [391]:
clean_dm = d_m_csv.drop(columns=['Unnamed: 0'])

In [449]:
clean_dm.describe()

,outflow,outflow_dai_only,outflow_surplus_buffer,total_period
count,"9,528.00","9,528.00","9,456.00","9,528.00"
mean,"730,298,227.76","723,044,786.34","7,308,670.67","4,381,789,366.59"
std,"1,172,359,348.27","1,153,238,025.26","20,455,276.75","2,882,217,947.64"
min,0.00,0.00,"-5,674,219.65",0.00
25%,"107,656,317.16","107,636,256.59",0.00,"1,093,258,152.24"
50%,"323,980,629.04","323,980,629.04",0.00,"5,189,669,321.25"
75%,"628,916,174.70","628,916,174.70",0.00,"6,203,283,988.26"
max,"6,014,959,678.79","5,952,992,110.30","83,550,327.25","10,118,832,636.48"


In [67]:
dai_maturity_df = d_m_csv.pivot_table(
    index='dt',  # or 'period' if your DataFrame's time column is named 'period'
    columns='maturity',
    values=['outflow', 'outflow_dai_only', 'outflow_surplus_buffer'],
    aggfunc='sum'  # or any other aggregation function that fits your needs
)

# Flatten the MultiIndex columns (optional, for cleaner column names)
dai_maturity_df.columns = ['_'.join(col).strip() for col in dai_maturity_df.columns.values]

# Reset the index if you want 'dt' back as a regular column
dai_maturity_df.reset_index(inplace=True)



In [392]:
dai_maturity_df.tail()

,outflow_1-block,outflow_1-day,outflow_1-month,outflow_1-week,outflow_1-year,outflow_3-months,outflow_dai_only_1-block,outflow_dai_only_1-day,outflow_dai_only_1-month,outflow_dai_only_1-week,outflow_dai_only_1-year,outflow_dai_only_3-months,outflow_surplus_buffer_1-block,outflow_surplus_buffer_1-day,outflow_surplus_buffer_1-month,outflow_surplus_buffer_1-week,outflow_surplus_buffer_1-year,outflow_surplus_buffer_3-months
dt,,,,,,,,,,,,,,,,,,
2024-03-16 00:00:00+00:00,"453,757,132.17","453,757,132.17","136,277,964.30","591,913,771.82","2,850,066,310.40","201,065,848.96","453,757,132.17","453,757,132.17","136,277,964.30","591,913,771.82","2,781,519,501.85","201,065,848.96",0.00,0.00,0.00,0.00,"68,546,808.55",0.00
2024-03-17 00:00:00+00:00,"455,307,007.70","455,307,007.70","137,425,908.66","596,032,698.73","2,849,349,624.78","202,759,537.36","455,307,007.70","455,307,007.70","137,425,908.66","596,032,698.73","2,780,461,988.11","202,759,537.36",0.00,0.00,0.00,0.00,"68,887,636.67",0.00
2024-03-18 00:00:00+00:00,"461,095,932.81","461,095,932.81","140,439,203.63","607,501,207.95","2,863,252,281.53","207,205,382.40","461,095,932.81","461,095,932.81","140,439,203.63","607,501,207.95","2,796,227,598.03","207,205,382.40",0.00,0.00,0.00,0.00,"67,024,683.50",0.00
2024-03-19 00:00:00+00:00,"455,524,838.32","455,524,838.32","137,117,144.96","595,167,015.97","2,854,314,452.33","202,303,984.36","455,524,838.32","455,524,838.32","137,117,144.96","595,167,015.97","2,787,586,179.26","202,303,984.36",0.00,0.00,0.00,0.00,"66,728,273.07",0.00
2024-03-20 00:00:00+00:00,"451,479,837.89","451,479,837.89","135,641,898.57","589,090,233.01","2,833,931,375.49","200,127,391.33","451,479,837.89","451,479,837.89","135,641,898.57","589,090,233.01","2,766,819,019.01","200,127,391.33",0.00,0.00,0.00,0.00,"67,112,356.48",0.00


In [68]:
dai_maturity_df['dt'] = pd.to_datetime(dai_maturity_df['dt'])
dai_maturity_df.set_index('dt', inplace=True)


start_date = dai_maturity_df.index.min()
end_date = dai_maturity_df.index.max()
date_range = pd.date_range(start=start_date, end=end_date, freq='D')  # 'D' for daily frequency




In [69]:
dai_maturity_df_reindexed = dai_maturity_df.reindex(date_range)

# Forward-fill missing values
dai_maturity_df_reindexed.ffill(inplace=True)

# Optionally, reset the index if you want 'dt' back as a column
dai_maturity_df_reindexed.reset_index(inplace=True)
dai_maturity_df_reindexed.rename(columns={'index': 'day'}, inplace=True)

In [70]:
dai_maturity_df_reindexed.columns = [f'dai_maturity_{col}' if col != 'day' and not col.startswith('dai_maturity_') else col for col in dai_maturity_df_reindexed.columns]

In [71]:
dai_maturity_df_reindexed.tail()

,day,dai_maturity_outflow_1-block,dai_maturity_outflow_1-day,dai_maturity_outflow_1-month,dai_maturity_outflow_1-week,dai_maturity_outflow_1-year,dai_maturity_outflow_3-months,dai_maturity_outflow_dai_only_1-block,dai_maturity_outflow_dai_only_1-day,dai_maturity_outflow_dai_only_1-month,dai_maturity_outflow_dai_only_1-week,dai_maturity_outflow_dai_only_1-year,dai_maturity_outflow_dai_only_3-months,dai_maturity_outflow_surplus_buffer_1-block,dai_maturity_outflow_surplus_buffer_1-day,dai_maturity_outflow_surplus_buffer_1-month,dai_maturity_outflow_surplus_buffer_1-week,dai_maturity_outflow_surplus_buffer_1-year,dai_maturity_outflow_surplus_buffer_3-months
1585,2024-03-16 00:00:00+00:00,"453,757,132.17","453,757,132.17","136,277,964.30","591,913,771.82","2,850,066,310.40","201,065,848.96","453,757,132.17","453,757,132.17","136,277,964.30","591,913,771.82","2,781,519,501.85","201,065,848.96",0.00,0.00,0.00,0.00,"68,546,808.55",0.00
1586,2024-03-17 00:00:00+00:00,"455,307,007.70","455,307,007.70","137,425,908.66","596,032,698.73","2,849,349,624.78","202,759,537.36","455,307,007.70","455,307,007.70","137,425,908.66","596,032,698.73","2,780,461,988.11","202,759,537.36",0.00,0.00,0.00,0.00,"68,887,636.67",0.00
1587,2024-03-18 00:00:00+00:00,"461,095,932.81","461,095,932.81","140,439,203.63","607,501,207.95","2,863,252,281.53","207,205,382.40","461,095,932.81","461,095,932.81","140,439,203.63","607,501,207.95","2,796,227,598.03","207,205,382.40",0.00,0.00,0.00,0.00,"67,024,683.50",0.00
1588,2024-03-19 00:00:00+00:00,"455,524,838.32","455,524,838.32","137,117,144.96","595,167,015.97","2,854,314,452.33","202,303,984.36","455,524,838.32","455,524,838.32","137,117,144.96","595,167,015.97","2,787,586,179.26","202,303,984.36",0.00,0.00,0.00,0.00,"66,728,273.07",0.00
1589,2024-03-20 00:00:00+00:00,"451,479,837.89","451,479,837.89","135,641,898.57","589,090,233.01","2,833,931,375.49","200,127,391.33","451,479,837.89","451,479,837.89","135,641,898.57","589,090,233.01","2,766,819,019.01","200,127,391.33",0.00,0.00,0.00,0.00,"67,112,356.48",0.00


MakerDAO Stablecoin Ratio
This can give insights into the proportion of assets held in stablecoins (including DAI) relative to other assets. A higher stablecoin ratio might suggest a preference for stability within the MakerDAO system, which can have implications for DAI's stability.

In [72]:
#stablecoin_ratio_df = fetch_dune_data(58136)

In [73]:
#stablecoin_ratio_df['dt'] = pd.to_datetime(stablecoin_ratio_df['dt'])
#stablecoin_ratio_df.set_index('dt', inplace=True)

In [74]:
#stablecoin_ratio_df.to_csv(stablecoin_ratio_path)

In [75]:
stablecoin_ratio_path = '../data/csv/srp.csv'

In [76]:
stablecoin_ratio_csv = pd.read_csv(stablecoin_ratio_path, index_col='dt', parse_dates=True)

In [77]:
stablecoin_ratio_csv.head()

,Unnamed: 0,,assets_size,assets_stablecoins,assets_usdc,stablecoins_ratio,usdc_ratio
dt,,,,,,,
2024-03-21 00:00:00+00:00,0,1.00,"4,692,077,547.49","510,492,745.88","509,792,133.98",0.11,0.11
2024-03-20 00:00:00+00:00,1,1.00,"4,665,516,334.77","451,367,053.87","450,666,441.97",0.10,0.10
2024-03-19 00:00:00+00:00,2,1.00,"4,703,718,034.84","404,362,171.82","403,655,435.55",0.09,0.09
2024-03-18 00:00:00+00:00,3,1.00,"4,744,355,701.71","399,931,319.66","399,224,583.39",0.08,0.08
2024-03-17 00:00:00+00:00,4,1.00,"4,699,947,545.51","632,391,001.19","631,684,264.92",0.13,0.13


In [78]:
stable_coin_ratios = stablecoin_ratio_csv[['stablecoins_ratio','usdc_ratio']]

In [79]:
stable_coin_ratios = stable_coin_ratios.rename_axis('day')

In [80]:
stable_coin_ratios.index

DatetimeIndex(['2024-03-21 00:00:00+00:00', '2024-03-20 00:00:00+00:00',
               '2024-03-19 00:00:00+00:00', '2024-03-18 00:00:00+00:00',
               '2024-03-17 00:00:00+00:00', '2024-03-16 00:00:00+00:00',
               '2024-03-15 00:00:00+00:00', '2024-03-14 00:00:00+00:00',
               '2024-03-13 00:00:00+00:00', '2024-03-12 00:00:00+00:00',
               ...
               '2019-11-27 00:00:00+00:00', '2019-11-26 00:00:00+00:00',
               '2019-11-25 00:00:00+00:00', '2019-11-24 00:00:00+00:00',
               '2019-11-23 00:00:00+00:00', '2019-11-22 00:00:00+00:00',
               '2019-11-21 00:00:00+00:00', '2019-11-20 00:00:00+00:00',
               '2019-11-19 00:00:00+00:00', '2019-11-18 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='day', length=1586, freq=None)

Maker Peg Stability Module Stats
Data on the Peg Stability Module, which helps maintain DAI's peg to the USD, can be vital. Insights into the inflows, outflows, and balances within the PSM can directly indicate efforts to stabilize DAI.

In [81]:
#psm_stats_df = fetch_dune_data(17216)

In [82]:
#psm_stats_df['date'] = pd.to_datetime(psm_stats_df['date'])

In [83]:
psm_stats_path = '../data/csv/psm.csv'

In [84]:
#psm_stats_df.to_csv(psm_stats_path)

In [85]:
psm_stats_csv = pd.read_csv(psm_stats_path, index_col='date', parse_dates=True)

In [86]:
psm_stats_csv.columns = [f'psm_{col}' if col != 'dt' and not col.startswith('psm_') else col for col in psm_stats_csv.columns]


In [87]:
psm_stats_csv = psm_stats_csv.drop(columns=['psm_Unnamed: 0'])

In [88]:
psm_stats_csv = psm_stats_csv.rename_axis('day')

In [403]:
psm_stats_csv.columns

Index(['psm_change', 'psm_change_excl_rwa', 'psm_change_excl_rwa_30d_avg',
       'psm_change_excl_rwa_7d_avg', 'psm_fees', 'psm_inflow',
       'psm_inflow_exl_rwa', 'psm_lifetime_fees', 'psm_lifetime_turnover',
       'psm_outflow', 'psm_balance', 'psm_turnover'],
      dtype='object')

In [443]:
psm_stats_csv[['psm_balance','psm_fees','psm_inflow','psm_outflow','psm_turnover']].describe()

,psm_balance,psm_fees,psm_inflow,psm_outflow,psm_turnover
count,"1,179.00","1,102.00","1,102.00","1,102.00","1,102.00"
mean,"2,495,919,311.99","4,037.44","45,752,972.38","45,289,730.32","91,042,702.70"
std,"1,628,328,770.12","17,575.39","71,460,076.01","62,717,895.80","116,889,284.80"
min,1.00,0.00,0.00,0.00,1.00
25%,"673,188,300.21",0.00,"12,117,870.98","9,329,900.97","28,952,809.74"
50%,"3,041,213,026.50",0.00,"26,787,127.56","29,106,242.25","59,685,795.04"
75%,"3,652,875,373.77",0.32,"54,102,189.62","60,721,834.75","114,257,782.07"
max,"5,965,535,187.98","329,095.83","1,399,943,587.76","828,699,035.76","2,005,261,976.79"


Where is dai lets us know how dai is being used; dai in lending could be considered to add to stability 

In [90]:
#where_is_dai_df = fetch_dune_data(54599)

In [91]:
#where_is_dai_df['dt'] = pd.to_datetime(where_is_dai_df['dt'])

In [92]:
wid_path = '../data/csv/wid.csv'

In [93]:
#where_is_dai_df.to_csv(wid_path)

In [94]:
where_is_dai_csv = pd.read_csv(wid_path, index_col='dt', parse_dates=True)

In [95]:
where_is_dai_csv = where_is_dai_csv.rename_axis('day')

In [406]:
print(where_is_dai_csv.describe())

       where_is_dai_balance  where_is_dai_total_balance
count             11,112.00                   11,112.00
mean         623,630,370.48            4,989,042,963.87
std          885,053,602.91            2,513,963,972.77
min                    0.00              113,631,008.03
25%           37,929,286.64            4,426,586,805.76
50%          252,756,050.89            5,195,896,461.42
75%          686,266,921.16            6,378,627,017.74
max        4,078,699,922.32           10,080,785,059.41


In [96]:
where_is_dai_csv.columns = [f'where_is_dai_{col}' if col != 'day' and not col.startswith('where_is_dai_') else col for col in where_is_dai_csv.columns]

In [97]:
where_is_dai_csv = where_is_dai_csv.drop(columns=['where_is_dai_Unnamed: 0'])

In [98]:
where_is_dai_csv.shape[0]

11112

In [99]:
where_is_dai_csv_table = where_is_dai_csv.pivot_table(values='where_is_dai_balance', index='day', columns='where_is_dai_wallet', aggfunc='sum')

In [100]:
where_is_dai_csv_table.describe()

where_is_dai_wallet,Bridge,CeFi,Dai Savings,Dex,EOA,Lending,Other,Treasury
count,"1,389.00","1,389.00","1,389.00","1,389.00","1,389.00","1,389.00","1,389.00","1,389.00"
mean,"519,571,584.34","97,793,318.99","311,612,990.83","759,685,160.26","2,313,033,791.12","583,815,304.12","384,965,889.61","18,564,924.60"
std,"479,488,038.35","78,415,911.28","518,564,427.53","660,249,624.37","1,133,804,050.66","565,104,617.62","199,111,750.50","22,963,663.97"
min,91.54,"6,032,365.70","1,167,096.23","5,969,545.93","70,539,903.52","2,382,557.63","7,624,088.20",0.00
25%,"254,354,292.48","22,517,650.67","12,215,114.08","242,906,003.63","1,797,171,880.58","86,534,442.33","256,075,295.20",0.00
50%,"452,459,560.65","83,466,578.34","90,963,507.53","563,571,678.17","2,869,541,236.00","379,804,255.54","384,559,660.50","19,322,535.41"
75%,"611,193,769.11","161,210,994.77","253,531,898.08","1,035,302,796.12","3,126,200,483.96","1,051,192,009.32","539,753,467.21","27,500,346.82"
max,"2,131,833,654.66","311,873,211.38","1,750,756,886.52","2,603,174,883.49","4,078,699,922.32","1,854,438,972.60","829,237,365.55","155,646,703.53"


In [101]:
where_is_dai_csv_table.columns = [f'where_is_dai_{col}' if col != 'day' and not col.startswith('where_is_dai_') else col for col in where_is_dai_csv_table.columns]

In [102]:
where_is_dai_csv_table.shape[0]

1389

Daily surplus buffer
Provides information on the surplus buffer in MakerDAO, which is a key financial metric. The surplus buffer acts as a reserve to cover potential system shortfalls and ensures the stability and solvency of the system. This data could be valuable for understanding the financial health and risk management strategies of MakerDAO over time

In [103]:
#daily_surplus_buffer = fetch_dune_data(3567837)

In [104]:
#daily_surplus_buffer['period'] = pd.to_datetime(daily_surplus_buffer['period'])

In [105]:
dsb_path = '../data/csv/dsb.csv'
#daily_surplus_buffer.to_csv(dsb_path)
daily_surplus_buffer_csv = pd.read_csv(dsb_path, index_col='period', parse_dates=True)

In [106]:
daily_surplus_buffer_csv = daily_surplus_buffer_csv.drop(columns=['Unnamed: 0'])

In [444]:
daily_surplus_buffer_csv.describe()

,daily_surplus_buffer_delta_30d,daily_surplus_buffer_delta_90d,daily_surplus_buffer
count,"1,609.00","1,609.00","1,609.00"
mean,"11,130,093.66","10,451,707.83","42,392,130.19"
std,"52,938,741.60","38,867,971.04","30,073,000.80"
min,"-210,344,528.52","-93,067,840.04","-5,674,219.91"
25%,"-10,263,499.22","-9,109,403.83","4,001,728.36"
50%,"2,130,021.51","2,024,616.70","50,260,545.40"
75%,"41,086,534.67","30,897,105.18","69,818,697.29"
max,"154,842,096.11","122,533,492.58","83,553,170.97"


In [108]:
daily_surplus_buffer_csv.columns = [f'daily_surplus_buffer_{col}' if col != 'period' and not col.startswith('surplus_buffer') else col for col in daily_surplus_buffer_csv.columns]

In [109]:
daily_surplus_buffer_csv = daily_surplus_buffer_csv.rename(columns={'surplus_buffer':'daily_surplus_buffer'})


In [110]:
daily_surplus_buffer_csv = daily_surplus_buffer_csv.rename_axis('day')

In [111]:
daily_surplus_buffer_csv.head()

,daily_surplus_buffer_delta_30d,daily_surplus_buffer_delta_90d,daily_surplus_buffer
day,,,
2024-03-27 00:00:00+00:00,"-118,286,288.53","-23,073,880.40","42,679,254.97"
2024-03-26 00:00:00+00:00,"-117,968,788.93","-22,860,798.30","42,972,541.24"
2024-03-25 00:00:00+00:00,"-114,559,431.40","-21,963,371.46","43,449,261.66"
2024-03-24 00:00:00+00:00,"-110,361,924.94","-20,375,056.07","44,097,497.35"
2024-03-23 00:00:00+00:00,"-105,856,419.40","-19,278,637.62","44,733,525.24"


Stability Fee history and Vault History - Rates Set by DAO
dart = debt balance

In [112]:
#sf_df = fetch_dune_data(3551110)

In [113]:
#sf_df['period'] = pd.to_datetime(sf_df['period'])

In [114]:
sf_path = '../data/csv/sf.csv'
#sf_df.to_csv(sf_path)
sf_history_csv = pd.read_csv(sf_path, index_col='period', parse_dates=True)

In [115]:
sf_history_csv

,Unnamed: 0,annualized,annualized_revenues,dart,ilk,total_ann_revenues
period,,,,,,
2024-03-21 00:00:00+00:00,0,0.05,0.00,0.00,GNO-A,"176,643,660.32"
2024-03-21 00:00:00+00:00,1,0.00,"42,545.93","70,910,068.44",GUNIV3DAIUSDC2-A,"176,643,660.32"
2024-03-21 00:00:00+00:00,2,0.03,-0.00,-0.00,LINK-A,"176,643,660.32"
2024-03-21 00:00:00+00:00,3,0.01,-0.00,-0.00,YFI-A,"176,643,660.32"
2024-03-21 00:00:00+00:00,4,0.01,0.00,0.00,COMP-A,"176,643,660.32"
...,...,...,...,...,...,...
2019-11-01 00:00:00+00:00,95411,NaN,NaN,NaN,WBTC-A,NaN
2019-11-01 00:00:00+00:00,95412,NaN,NaN,NaN,UNIV2UNIETH-A,NaN
2019-11-01 00:00:00+00:00,95413,NaN,NaN,NaN,RWA005-A,NaN


In [116]:
sf_history_csv_reset = sf_history_csv.reset_index()

In [117]:
sf_history_csv_clean = sf_history_csv_reset.drop_duplicates(subset=['period', 'ilk'], keep='last')

In [118]:
sf_history_csv_clean = sf_history_csv_clean.drop(columns='Unnamed: 0')

In [119]:
sf_history_csv_clean['period'] = sf_history_csv_clean['period'].dt.date

In [120]:
sf_history_csv_clean = sf_history_csv_clean.rename(columns={'period':'day'})

DAI Savings Rate Historical - Set by DAO

In [121]:
#dsr_rate = fetch_dune_data(3581248)

In [122]:
dsr_rate_path = '../data/csv/dsr.csv'
#dsr_rate.to_csv(dsr_rate_path)
dsr_rate_csv = pd.read_csv(dsr_rate_path, index_col='dt', parse_dates=True)

In [123]:
dsr_rate_csv['dsr_rate'].describe()

count   1,552.00
mean        0.01
std         0.03
min         0.00
25%         0.00
50%         0.00
75%         0.01
max         0.15
Name: dsr_rate, dtype: float64

In [124]:
dsr_rate_csv = dsr_rate_csv.drop(columns=['Unnamed: 0'])

In [125]:
dsr_rate_csv['dai_percent_in_dsr'] = dsr_rate_csv['dsr_balance'] / dsr_rate_csv['total_balance']
dsr_rate_csv['dai_percent_out_dsr'] = dsr_rate_csv['non_dsr_balance'] / dsr_rate_csv['total_balance']

In [126]:
dsr_rate_csv = dsr_rate_csv.rename_axis('day')

In [408]:
print(dsr_rate_csv.describe())

           dsr_balance  dsr_rate  non_dsr_balance     total_balance  \
count         1,552.00  1,552.00         1,552.00          1,552.00   
mean    293,268,678.28      0.01 4,181,101,606.37  4,474,370,284.65   
std     508,781,220.45      0.03 2,795,421,701.77  2,786,126,412.57   
min       1,167,096.23      0.00    42,257,176.08     72,614,957.58   
25%      13,831,047.69      0.00 1,494,884,857.88  1,501,681,100.28   
50%      78,372,158.21      0.00 4,361,860,196.87  5,130,678,028.50   
75%     208,537,359.71      0.01 6,149,889,671.04  6,183,615,643.32   
max   1,750,756,886.52      0.15 9,944,328,573.43 10,056,865,067.98   

       dai_percent_in_dsr  dai_percent_out_dsr  
count            1,552.00             1,552.00  
mean                 0.13                 0.87  
std                  0.16                 0.16  
min                  0.00                 0.38  
25%                  0.01                 0.72  
50%                  0.02                 0.98  
75%              

dsr flows

In [128]:
#dsr_flows = fetch_dune_data(1753750)

In [129]:
dsr_flows_path='../data/csv/dsr_flows.csv'
#dsr_flows.to_csv(dsr_flows_path)
dsr_flows_csv = pd.read_csv(dsr_flows_path, index_col='period', parse_dates=True)

In [130]:
dsr_flows_csv.shape[0]

581

In [131]:
dsr_flows_csv = dsr_flows_csv.drop(columns=['Unnamed: 0','balance'])
dsr_flows_csv = dsr_flows_csv.rename_axis('day')

In [132]:
dsr_flows_csv.columns = [f'dsr_{col}' if col != 'day' and not col.startswith('surplus_buffer') else col for col in dsr_flows_csv.columns]

In [133]:
dsr_df = dsr_flows_csv.merge(dsr_rate_csv, on=['day'], how='inner')

In [134]:
dsr_df = dsr_df.rename(columns={'total_balance':'dai_total_balance'})
dsr_df = dsr_df.rename(columns={'non_dsr_balance':'dai_circulating'})

In [135]:
dsr_df.describe()

,dsr_inflow,dsr_interest,dsr_outflow,dsr_balance,dsr_rate,dai_circulating,dai_total_balance,dai_percent_in_dsr,dai_percent_out_dsr
count,199.00,420.00,126.00,581.00,581.00,581.00,581.00,581.00,581.00
mean,"18,211,028.03","128,169.53","17,498,729.02","632,212,219.96",0.03,"4,584,444,424.95","5,216,656,644.91",0.12,0.88
std,"34,118,088.22","126,667.16","43,686,010.07","689,782,183.06",0.03,"895,024,557.86","495,795,966.59",0.13,0.13
min,0.00,0.12,0.01,"1,167,096.23",0.00,"3,182,053,005.07","4,336,565,083.61",0.00,0.67
25%,"1,995,736.75","3,895.16","625,555.75","35,775,627.77",0.00,"3,756,055,150.83","4,854,191,351.00",0.01,0.72
50%,"8,698,729.54","156,761.36","6,453,848.96","109,586,897.55",0.01,"4,542,041,297.79","5,174,925,449.48",0.02,0.98
75%,"19,085,676.64","210,194.81","14,916,349.44","1,434,744,378.05",0.05,"5,136,076,330.01","5,391,627,556.52",0.28,0.99
max,"298,378,513.73","591,988.67","337,868,152.03","1,750,756,886.52",0.15,"6,549,164,445.71","6,550,365,936.14",0.33,1.00


In [136]:
#cum_bal_and_safetyprice_and_safetyvalue = vault_stats_6_20_through_6_21 

In [137]:
cumbal_stats_path ='../data/csv/cumbal.csv'
#cum_bal_and_safetyprice_and_safetyvalue.to_csv(cumbal_stats_path)
cumbal_csv = pd.read_csv(cumbal_stats_path, index_col = 'day', parse_dates=True)

In [138]:
#debtbal_lpenalty_lratio = fetch_dune_data(3568425)

In [139]:
debtbal_lpenalty_lratio_path = '../data/csv/debtbal_lpenalty_lratio.csv'
#debtbal_lpenalty_lratio.to_csv(debtbal_lpenalty_lratio_path)
debtbal_lpenalty_lratio_path_csv = pd.read_csv(debtbal_lpenalty_lratio_path, index_col = 'day', parse_dates=True)

In [140]:
debtbal_lpenalty_lratio_path_csv.head()

,Unnamed: 0,debt_balance,ilk,liquidation_penalty,liquidation_ratio
day,,,,,
2024-03-27,0,0.00,BAL-A,0.00,7.48
2024-03-27,1,-0.00,KNC-A,0.00,12.28
2024-03-27,2,"300,484,322.01",ETH-C,0.13,1.70
2024-03-27,3,0.00,GUSD-A,0.13,15.00
2024-03-27,4,0.00,BAT-A,0.00,47.50


In [141]:
debtbal_lpenalty_lratio_path_csv_reset = debtbal_lpenalty_lratio_path_csv.reset_index()
debtbal_lpenalty_lratio_path_csv_clean = debtbal_lpenalty_lratio_path_csv_reset.drop_duplicates(subset=['day', 'ilk'], keep='last')

In [142]:
debtbal_lpenalty_lratio_path_csv_clean = debtbal_lpenalty_lratio_path_csv_clean.drop(columns='Unnamed: 0')

In [143]:
#dceiling_dfloor_scratio = fetch_dune_data(3568438)

In [144]:
dceiling_dfloor_scratio_path = '../data/csv/dceiling_dfloor_scratio.csv'
#dceiling_dfloor_scratio.to_csv(dceiling_dfloor_scratio_path)
dceiling_dfloor_scratio_csv = pd.read_csv(dceiling_dfloor_scratio_path, index_col = 'day', parse_dates=True)

In [145]:
#vault_market_price = fetch_dune_data(3568453)

In [146]:
vault_market_price_path = '../data/csv/vault_market_price.csv'
#vault_market_price.to_csv(vault_market_price_path)
vault_market_price_csv = pd.read_csv(vault_market_price_path, index_col = 'day', parse_dates=True)

In [147]:
vault_market_price_csv.head()

,Unnamed: 0,ilk,market_price
day,,,
2024-03-27,0,GUNIV3DAIUSDC2-A,203.39
2024-03-27,1,BAL-A,21.64
2024-03-27,2,LINK-A,6.28
2024-03-27,3,UNIV2WBTCDAI-A,"31,062,885.00"
2024-03-27,4,WBTC-A,"69,767.86"


In [148]:
vault_market_price_csv_reset = vault_market_price_csv.reset_index()
vault_market_price_csv_clean = vault_market_price_csv_reset.drop_duplicates(subset=['day', 'ilk'], keep='last')

In [149]:
vault_market_price_csv_clean = vault_market_price_csv_clean.drop(columns=['Unnamed: 0'])

In [150]:
cumbal_csv_reset = cumbal_csv.reset_index()
dceiling_dfloor_scratio_csv_reset = dceiling_dfloor_scratio_csv.reset_index()

In [151]:
cumbal_csv_clean = cumbal_csv_reset.drop_duplicates(subset=['day', 'ilk'], keep='last')
dceiling_dfloor_scratio_csv_clean = dceiling_dfloor_scratio_csv_reset.drop_duplicates(subset=['day', 'ilk'], keep='last')

In [152]:
cumbal_csv_clean = cumbal_csv_clean.drop(columns=['Unnamed: 0'])

In [153]:
dceiling_dfloor_scratio_csv_clean = dceiling_dfloor_scratio_csv_clean.drop(columns=['Unnamed: 0'])

In [154]:
comprehensive_vault_stats = pd.merge(cumbal_csv_clean, dceiling_dfloor_scratio_csv_clean, on=['day', 'ilk'], how='inner')

In [155]:
comprehensive_vault_stats = comprehensive_vault_stats.merge(vault_market_price_csv_clean, on=['day', 'ilk'], how='inner' )

In [156]:
comprehensive_vault_stats = comprehensive_vault_stats.merge(debtbal_lpenalty_lratio_path_csv_clean, on=['day', 'ilk'], how='inner')

In [157]:
comprehensive_vault_stats['day'] = pd.to_datetime(comprehensive_vault_stats['day'])
sf_history_csv_clean['day'] = pd.to_datetime(sf_history_csv_clean['day'])

In [158]:
ir_csv.columns

Index(['ilk', 'daily_revenues'], dtype='object')

In [159]:
comprehensive_vault_stats.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio'],
      dtype='object')

In [160]:
comprehensive_vault_stats = comprehensive_vault_stats.merge(ir_csv, on=['day', 'ilk'], how='inner')

In [161]:
comprehensive_vault_stats = comprehensive_vault_stats.merge(sf_history_csv_clean, on=['day', 'ilk'], how='inner')

In [163]:
comprehensive_vault_stats['day'] = comprehensive_vault_stats['day'].dt.tz_convert('UTC')
dai_maturity_df_reindexed['day'] = dai_maturity_df_reindexed['day'].dt.tz_convert('UTC')

In [164]:
# Ensure both 'day' columns are timezone-aware and in UTC
comprehensive_vault_stats['day'] = comprehensive_vault_stats['day'].dt.tz_convert('UTC')
dai_maturity_df_reindexed['day'] = dai_maturity_df_reindexed['day'].dt.tz_convert('UTC')


In [165]:
stable_coin_ratios.index

DatetimeIndex(['2024-03-21 00:00:00+00:00', '2024-03-20 00:00:00+00:00',
               '2024-03-19 00:00:00+00:00', '2024-03-18 00:00:00+00:00',
               '2024-03-17 00:00:00+00:00', '2024-03-16 00:00:00+00:00',
               '2024-03-15 00:00:00+00:00', '2024-03-14 00:00:00+00:00',
               '2024-03-13 00:00:00+00:00', '2024-03-12 00:00:00+00:00',
               ...
               '2019-11-27 00:00:00+00:00', '2019-11-26 00:00:00+00:00',
               '2019-11-25 00:00:00+00:00', '2019-11-24 00:00:00+00:00',
               '2019-11-23 00:00:00+00:00', '2019-11-22 00:00:00+00:00',
               '2019-11-21 00:00:00+00:00', '2019-11-20 00:00:00+00:00',
               '2019-11-19 00:00:00+00:00', '2019-11-18 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='day', length=1586, freq=None)

In [166]:
comprehensive_vault_stats.tail()

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,liquidation_penalty,liquidation_ratio,daily_revenues,annualized,annualized_revenues,dart,total_ann_revenues
21747,2019-11-15 00:00:00+00:00,0.42,ETH-A,119.74,50.29,0.00,20.00,0.00,NaN,NaN,NaN,NaN,0.00,0.04,NaN,NaN,NaN
21748,2019-11-15 00:00:00+00:00,10.00,BAT-A,0.17,1.66,0.00,20.00,0.00,NaN,NaN,NaN,NaN,0.00,0.04,NaN,NaN,NaN
21749,2019-11-13 00:00:00+00:00,NaN,SAI,0.00,NaN,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN
21750,2019-11-13 00:00:00+00:00,0.42,ETH-A,0.00,0.00,0.00,20.00,0.00,NaN,NaN,NaN,NaN,0.00,0.04,NaN,NaN,NaN
21751,2019-11-13 00:00:00+00:00,10.00,BAT-A,0.00,0.00,0.00,20.00,0.00,NaN,NaN,NaN,NaN,0.00,0.04,NaN,NaN,NaN


In [167]:
comprehensive_vault_stats.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized', 'annualized_revenues', 'dart', 'total_ann_revenues'],
      dtype='object')

In [168]:
comprehensive_vault_stats = comprehensive_vault_stats.rename(columns={'annualized':'annualized stability fee'})

In [169]:
def determine_status(row):
    # The vault is considered closed if 'dai_ceiling' is 0
    # This includes the first day if 'dai_ceiling' starts at 0 or if it drops to 0 from a nonzero value
    if pd.isnull(row['prev_dai_ceiling']) and row['dai_ceiling'] == 0:
        return 'Closed'
    elif row['prev_dai_ceiling'] >= 0 and row['dai_ceiling'] == 0:
        return 'Closed'
    # Check if 'safety_price' is 0
    elif row['safety_price'] == 0:
        return 'Closed'
    else:
        return 'Open'

In [170]:
comprehensive_vault_stats['market_price'] = np.where(comprehensive_vault_stats['liquidation_ratio'].isnull(), comprehensive_vault_stats['safety_price'], comprehensive_vault_stats['market_price'])

In [171]:
comprehensive_vault_stats = comprehensive_vault_stats.sort_values('day')
comprehensive_vault_stats['prev_dai_ceiling'] = comprehensive_vault_stats.groupby('ilk')['dai_ceiling'].shift(1)

In [172]:
comprehensive_vault_stats['status'] = comprehensive_vault_stats.apply(determine_status, axis=1)


In [173]:
comprehensive_vault_stats.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized stability fee', 'annualized_revenues', 'dart',
       'total_ann_revenues', 'prev_dai_ceiling', 'status'],
      dtype='object')

In [174]:
comprehensive_vault_stats['market_collateral_ratio'] = np.where(comprehensive_vault_stats['status']=='Open',((comprehensive_vault_stats['usd_safety_value'] *comprehensive_vault_stats['liquidation_ratio']) / comprehensive_vault_stats['debt_balance']), np.nan)
comprehensive_vault_stats['market_collateral_ratio'] = np.where(comprehensive_vault_stats['debt_balance']==0,0, comprehensive_vault_stats['market_collateral_ratio'])

comprehensive_vault_stats['safety_collateral_ratio'] = np.where(comprehensive_vault_stats['status']=='Open',comprehensive_vault_stats['safety_collateral_ratio'], np.nan)
comprehensive_vault_stats['safety_collateral_ratio'] = np.where(comprehensive_vault_stats['status'] == 'Closed', 0, comprehensive_vault_stats['safety_collateral_ratio'])
comprehensive_vault_stats['safety_collateral_ratio'] = np.where(comprehensive_vault_stats['debt_balance'] <= 0, 0, comprehensive_vault_stats['safety_collateral_ratio'])
comprehensive_vault_stats[['annualized_revenues', 'dart']] = np.where(comprehensive_vault_stats[['annualized_revenues', 'dart']] <= 0, 0, comprehensive_vault_stats[['annualized_revenues', 'dart']])

comprehensive_vault_stats['collateral_usd'] = comprehensive_vault_stats['usd_safety_value'] * comprehensive_vault_stats['liquidation_ratio']
comprehensive_vault_stats['hypothetical_dai_ceiling'] = comprehensive_vault_stats['collateral_usd']* (comprehensive_vault_stats['liquidation_ratio'] / 2)

In [175]:
#replace values when vault is closed to 0?

In [176]:
def clean_small_values(value, threshold=1e-8):
    """
    Set small negative and positive values within a specified threshold to zero.

    Parameters:
    - value: The value to be cleaned.
    - threshold: Values within +/- this threshold will be set to zero.

    Returns:
    - The cleaned value.
    """
    if -threshold < value < threshold:
        return 0.0
    else:
        return value

# Apply this function to the entire DataFrame
comprehensive_vault_stats[['cumulative_collateral','usd_safety_value','collateral_usd','hypothetical_dai_ceiling','debt_balance']] = comprehensive_vault_stats[['cumulative_collateral','usd_safety_value','collateral_usd','hypothetical_dai_ceiling','debt_balance']].applymap(lambda x: clean_small_values(x))


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_10768\4189237013.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  comprehensive_vault_stats[['cumulative_collateral','usd_safety_value','collateral_usd','hypothetical_dai_ceiling','debt_balance']] = comprehensive_vault_stats[['cumulative_collateral','usd_safety_value','collateral_usd','hypothetical_dai_ceiling','debt_balance']].applymap(lambda x: clean_small_values(x))


In [177]:
comprehensive_vault_stats = comprehensive_vault_stats[comprehensive_vault_stats['ilk'] != 'SAI']

UNIV2ETHUSDT-A is an outlier and will be removed from dataset.  

In [178]:
comprehensive_vault_stats[comprehensive_vault_stats['safety_collateral_ratio'] < 0 ]

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,...,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,status,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
12071,2021-08-08 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"137,635,170.15",0.00,"2,000,000.00","10,000.00",-12.95,"192,689,238.21",0.00,...,4.87,0.02,0.00,0.00,"38,032,976.74","2,006,530.97",Open,-18.13,0.00,0.00
12038,2021-08-09 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"139,168,140.36",0.00,"2,000,000.00","10,000.00",-13.09,"194,835,396.50",0.00,...,4.75,0.02,0.00,0.00,"38,121,293.97","2,000,000.00",Open,-18.33,0.00,0.00
11980,2021-08-10 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"140,063,017.89",0.00,"2,000,000.00","10,000.00",-13.18,"196,088,225.05",0.00,...,4.75,0.02,0.00,0.00,"38,691,927.83","2,000,000.00",Open,-18.45,0.00,0.00
11963,2021-08-11 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"141,384,297.67",0.00,"2,000,000.00","10,000.00",-13.30,"197,938,016.73",0.00,...,4.73,0.02,0.00,0.00,"38,773,987.68","2,000,000.00",Open,-18.62,0.00,0.00
11918,2021-08-12 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"137,867,966.00",0.00,"2,000,000.00","10,000.00",-12.97,"193,015,152.40",0.00,...,4.74,0.02,0.00,0.00,"40,333,600.87","2,000,000.00",Open,-18.16,0.00,0.00
11864,2021-08-13 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"141,650,852.35",0.00,"2,000,000.00","10,000.00",-13.33,"198,311,193.29",0.00,...,4.74,0.02,0.00,0.00,"39,855,556.08","2,000,000.00",Open,-18.66,0.00,0.00
11843,2021-08-14 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"142,284,082.28",0.00,"2,000,000.00","10,000.00",-13.39,"199,197,715.19",0.00,...,4.74,0.02,0.00,0.00,"40,109,465.52","2,000,000.00",Open,-18.74,0.00,0.00
11807,2021-08-15 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"139,997,139.81",0.00,"2,000,000.00","10,000.00",-13.17,"195,995,995.74",0.00,...,4.75,0.02,0.00,0.00,"40,161,597.87","2,000,000.00",Open,-18.44,0.00,0.00
11765,2021-08-16 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"142,001,139.88",0.00,"2,000,000.00","10,000.00",-13.36,"198,801,595.84",0.00,...,4.74,0.02,0.00,0.00,"40,275,499.78","2,000,000.00",Open,-18.70,0.00,0.00
11701,2021-08-17 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"140,896,250.11",0.00,"2,000,000.00","10,000.00",-13.26,"197,254,750.15",0.00,...,4.78,0.02,0.00,0.00,"40,012,479.05","2,000,000.00",Open,-18.56,0.00,0.00


In [179]:
comprehensive_vault_stats = comprehensive_vault_stats[comprehensive_vault_stats['ilk'] != 'UNIV2ETHUSDT-A']

In [180]:
comprehensive_vault_stats['debt_balance'].describe()

count          21,138.00
mean      138,522,602.01
std       387,265,830.49
min            -2,528.51
25%         1,403,187.58
50%        15,818,062.08
75%        88,412,143.20
max     3,361,006,014.83
Name: debt_balance, dtype: float64

In [181]:
#pd.set_option('display.max_columns', None)  # Show all columns
#pd.set_option('display.max_rows', None)  # Show all rows
#pd.set_option('display.max_colwidth', None)  # Show full content of each column

In [182]:
comprehensive_vault_stats['market_collateral_ratio'].describe()

count                    19,378.00
mean        160,876,285,659,620.75
std       7,377,037,911,158,396.00
min     -32,429,545,705,439,232.00
25%                           2.11
50%                           3.14
75%                           4.01
max     324,295,457,054,392,320.00
Name: market_collateral_ratio, dtype: float64

In [183]:
comprehensive_vault_stats['safety_collateral_ratio'].describe()

count                    21,501.00
mean        144,095,140,219,423.31
std       6,659,657,583,570,546.00
min                           0.00
25%                           1.41
50%                           1.94
75%                           2.50
max     308,852,816,242,278,400.00
Name: safety_collateral_ratio, dtype: float64

In [184]:
comprehensive_vault_stats[['day','ilk','market_collateral_ratio']][comprehensive_vault_stats['market_collateral_ratio'] > 3.31 ].sort_values('market_collateral_ratio')

,day,ilk,market_collateral_ratio
6980,2022-02-10 00:00:00+00:00,RENBTC-A,3.31
13243,2021-07-06 00:00:00+00:00,RENBTC-A,3.31
20676,2020-08-24 00:00:00+00:00,KNC-A,3.31
845,2023-12-07 00:00:00+00:00,ETH-B,3.31
2408,2023-05-19 00:00:00+00:00,RETH-A,3.31
...,...,...,...
7720,2022-01-05 00:00:00+00:00,RWA003-A,"324,295,457,054,392,320.00"
8489,2021-11-30 00:00:00+00:00,RWA003-A,"324,295,457,054,392,320.00"
8141,2021-12-18 00:00:00+00:00,RWA003-A,"324,295,457,054,392,320.00"
7463,2022-01-15 00:00:00+00:00,RWA003-A,"324,295,457,054,392,320.00"


In [185]:
comprehensive_vault_stats[['day','ilk','safety_collateral_ratio']][comprehensive_vault_stats['safety_collateral_ratio'] > 100 ]

,day,ilk,safety_collateral_ratio
15803,2021-04-21 00:00:00+00:00,RWA002-A,149.50
15727,2021-04-23 00:00:00+00:00,RWA001-A,"32,616,554.14"
15494,2021-04-30 00:00:00+00:00,RWA001-A,"32,616,554.14"
15464,2021-05-01 00:00:00+00:00,RWA001-A,"32,616,554.14"
15421,2021-05-02 00:00:00+00:00,RWA001-A,"32,616,554.14"
...,...,...,...
6015,2022-05-12 00:00:00+00:00,YFI-A,114.43
5992,2022-05-13 00:00:00+00:00,YFI-A,111.95
5637,2022-06-13 00:00:00+00:00,RWA003-A,113.28
4516,2022-09-30 00:00:00+00:00,RWA003-A,"9,651,650,507,571,200.00"


In [186]:
comprehensive_vault_stats[['day','ilk','debt_balance']][comprehensive_vault_stats['debt_balance'] < 0 ]

,day,ilk,debt_balance
3051,2023-03-11 00:00:00+00:00,DIRECT-AAVEV2-DAI,"-2,528.51"


In [187]:
comprehensive_vault_stats = comprehensive_vault_stats[comprehensive_vault_stats['ilk'] != 'DIRECT-AAVEV2-DAI']

In [188]:
comprehensive_vault_stats[['day','ilk','debt_balance']][comprehensive_vault_stats['debt_balance'] < 0 ]

,day,ilk,debt_balance


In [189]:
comprehensive_vault_stats['status'].describe()

count     21499
unique        2
top        Open
freq      19896
Name: status, dtype: object

In [190]:
comprehensive_vault_stats[['liquidation_ratio','liquidation_penalty','annualized stability fee',
       'annualized_revenues', 'dart', 'total_ann_revenues',
       'prev_dai_ceiling']].describe()

,liquidation_ratio,liquidation_penalty,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling
count,"21,004.00","15,665.00","21,493.00","21,136.00","21,136.00","21,491.00","21,445.00"
mean,1.49,-0.16,0.04,"3,167,088.52","138,541,671.93","72,830,970.89","173,188,464.60"
std,2.20,0.49,0.07,"10,267,730.84","387,325,542.99","62,927,201.18","470,887,247.03"
min,0.00,-1.00,0.00,0.00,0.00,767.83,0.00
25%,1.25,-1.00,0.01,"4,979.85","1,402,720.70","29,094,709.62","3,306,266.02"
50%,1.50,0.13,0.03,"172,814.73","15,890,305.95","52,913,177.34","20,000,000.00"
75%,1.75,0.13,0.05,"1,678,781.65","88,457,397.17","99,900,414.71","120,000,000.00"
max,100.00,0.13,0.50,"142,209,506.83","3,361,006,014.83","351,700,458.61","4,361,006,014.83"


In [191]:
closed_vaults = comprehensive_vault_stats[comprehensive_vault_stats['status'] == 'Closed']

In [192]:
zero_balances = comprehensive_vault_stats[comprehensive_vault_stats['cumulative_collateral'] == 0]

In [193]:
usdt_a = comprehensive_vault_stats[comprehensive_vault_stats['ilk']=='USDT-A']

In [194]:
usdt_a.head()

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,...,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,status,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
20599,2020-09-08 00:00:00+00:00,368.90,USDT-A,0.67,246.01,"10,000,000.00",100.00,1.00,1.00,245.99,...,0.02,0.06,14.76,245.99,"1,855,497.42",NaN,Open,1.50,369.02,276.76
20588,2020-09-09 00:00:00+00:00,0.00,USDT-A,0.67,0.00,"10,000,000.00",100.00,0.00,1.00,0.00,...,0.00,0.06,0.00,0.00,"1,763,209.76","10,000,000.00",Open,0.00,0.00,0.00
20567,2020-09-12 00:00:00+00:00,200.00,USDT-A,0.67,133.38,"10,000,000.00",100.00,0.00,1.00,0.00,...,0.12,0.06,0.00,0.00,"2,227,239.39","10,000,000.00",Open,0.00,200.07,150.05
20561,2020-09-13 00:00:00+00:00,"426,826.22",USDT-A,0.67,"284,680.54","10,000,000.00",100.00,1.50,1.00,"189,858.91",...,20.63,0.06,"11,391.53","189,858.91","2,223,945.55","10,000,000.00",Open,2.25,"427,020.81","320,265.61"
20558,2020-09-14 00:00:00+00:00,"531,480.11",USDT-A,0.67,"354,757.69","10,000,000.00",100.00,1.42,1.00,"249,920.00",...,44.86,0.06,"14,995.20","249,920.00","3,317,027.83","10,000,000.00",Open,2.13,"532,136.54","399,102.40"


In [195]:
zero_balances['ilk'].unique()

array(['USDT-A', 'UNIV2WBTCDAI-A', 'UNIV2DAIUSDT-A', 'RWA008-A'],
      dtype=object)

In [196]:
closed_vaults['ilk'].unique()

array(['BAT-A', 'ETH-A', 'LRC-A', 'COMP-A', 'LINK-A', 'ETH-B', 'BAL-A',
       'GUSD-A', 'TUSD-A', 'UNIV2DAIUSDC-A', 'UNIV2AAVEETH-A',
       'UNIV2WBTCDAI-A', 'UNIV2DAIUSDT-A', 'USDC-A', 'USDT-A', 'PAXUSD-A',
       'USDC-B', 'KNC-A', 'RWA006-A', 'GUNIV3DAIUSDC1-A',
       'UNIV2LINKETH-A', 'ZRX-A', 'AAVE-A', 'CRVV1ETHSTETH-A',
       'UNIV2DAIETH-A', 'UNIV2WBTCETH-A', 'UNI-A', 'RWA009-A', 'RETH-A',
       'RENBTC-A', 'GNO-A', 'GUNIV3DAIUSDC2-A', 'UNIV2USDCETH-A',
       'MATIC-A', 'RWA003-A', 'RWA005-A'], dtype=object)

In [197]:
wbtc = comprehensive_vault_stats[comprehensive_vault_stats['ilk']=='WBTC-A']
eth_a = comprehensive_vault_stats[comprehensive_vault_stats['ilk']=='ETH-A']

In [198]:
eth_a['status']

21750    Closed
21747    Closed
21745    Closed
21743    Closed
21742      Open
          ...  
37         Open
28         Open
18         Open
16         Open
7          Open
Name: status, Length: 1564, dtype: object

In [199]:
eth_a_df = eth_a['dai_ceiling'].to_frame('eth_a_dai_ceiling')

In [200]:
eth_a_zero_dai_ceiling = eth_a[eth_a['dai_ceiling'] == 0]

In [201]:
eth_a_zero_dai_ceiling.head()

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,...,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,status,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
21750,2019-11-13 00:00:00+00:00,0.42,ETH-A,0.00,0.00,0.00,20.00,0.00,0.00,NaN,...,0.00,0.04,NaN,NaN,NaN,NaN,Closed,NaN,NaN,NaN
21747,2019-11-15 00:00:00+00:00,0.42,ETH-A,119.74,50.29,0.00,20.00,0.00,119.74,NaN,...,0.00,0.04,NaN,NaN,NaN,0.00,Closed,NaN,NaN,NaN
21745,2019-11-16 00:00:00+00:00,0.42,ETH-A,121.69,51.11,0.00,20.00,0.00,121.69,NaN,...,0.00,0.04,NaN,NaN,NaN,0.00,Closed,NaN,NaN,NaN
21743,2019-11-17 00:00:00+00:00,0.43,ETH-A,123.31,53.02,0.00,20.00,0.00,123.31,NaN,...,0.00,0.04,NaN,NaN,NaN,0.00,Closed,NaN,NaN,NaN


In [202]:
eth_a_df.head()

,eth_a_dai_ceiling
21750,0.00
21747,0.00
21745,0.00
21743,0.00
21742,"50,000,000.00"


In [203]:
wbtc = wbtc.set_index('day')

In [204]:
comprehensive_vault_stats.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized stability fee', 'annualized_revenues', 'dart',
       'total_ann_revenues', 'prev_dai_ceiling', 'status',
       'market_collateral_ratio', 'collateral_usd',
       'hypothetical_dai_ceiling'],
      dtype='object')

In [205]:
comprehensive_vault_stats['ilk'].unique()

array(['BAT-A', 'ETH-A', 'USDC-A', 'WBTC-A', 'USDC-B', 'TUSD-A', 'ZRX-A',
       'KNC-A', 'MANA-A', 'USDT-A', 'PAXUSD-A', 'LRC-A', 'COMP-A',
       'LINK-A', 'ETH-B', 'BAL-A', 'YFI-A', 'GUSD-A', 'UNI-A', 'RENBTC-A',
       'AAVE-A', 'UNIV2DAIETH-A', 'PSM-USDC-A', 'UNIV2USDCETH-A',
       'UNIV2WBTCETH-A', 'UNIV2DAIUSDC-A', 'UNIV2LINKETH-A',
       'UNIV2UNIETH-A', 'UNIV2AAVEETH-A', 'UNIV2WBTCDAI-A',
       'UNIV2DAIUSDT-A', 'ETH-C', 'RWA002-A', 'RWA001-A', 'RWA003-A',
       'RWA005-A', 'RWA006-A', 'RWA004-A', 'MATIC-A', 'PSM-PAX-A',
       'GUNIV3DAIUSDC1-A', 'WSTETH-A', 'WBTC-B', 'WBTC-C', 'PSM-GUSD-A',
       'GUNIV3DAIUSDC2-A', 'CRVV1ETHSTETH-A', 'WSTETH-B', 'RWA009-A',
       'RETH-A', 'RWA008-A', 'GNO-A', 'RWA013-A', 'RWA012-A'],
      dtype=object)

In [206]:
comprehensive_vault_stats['status'].head()

21751    Closed
21750    Closed
21748    Closed
21747    Closed
21746    Closed
Name: status, dtype: object

In [207]:
no_nan_vaults = comprehensive_vault_stats.copy()
no_nan_vaults = no_nan_vaults.fillna(0)

In [409]:
no_nan_vaults.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized stability fee', 'annualized_revenues', 'dart',
       'total_ann_revenues', 'prev_dai_ceiling', 'status',
       'market_collateral_ratio', 'collateral_usd',
       'hypothetical_dai_ceiling'],
      dtype='object')

In [ ]:
print(no_nan_vaults[''].describe())

In [208]:
comprehensive_vault_stats.shape[0]

21499

In [415]:
comprehensive_vault_stats['safety_collateral_ratio'].describe()

count                    21,499.00
mean        144,108,545,041,993.62
std       6,659,967,211,665,183.00
min                           0.00
25%                           1.41
50%                           1.94
75%                           2.50
max     308,852,816,242,278,400.00
Name: safety_collateral_ratio, dtype: float64

In [210]:
comprehensive_vault_stats.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized stability fee', 'annualized_revenues', 'dart',
       'total_ann_revenues', 'prev_dai_ceiling', 'status',
       'market_collateral_ratio', 'collateral_usd',
       'hypothetical_dai_ceiling'],
      dtype='object')

In [211]:
comprehensive_vault_stats[comprehensive_vault_stats['ilk']=='ETH-A']

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,...,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,status,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
21750,2019-11-13 00:00:00+00:00,0.42,ETH-A,0.00,0.00,0.00,20.00,0.00,0.00,NaN,...,0.00,0.04,NaN,NaN,NaN,NaN,Closed,NaN,NaN,NaN
21747,2019-11-15 00:00:00+00:00,0.42,ETH-A,119.74,50.29,0.00,20.00,0.00,119.74,NaN,...,0.00,0.04,NaN,NaN,NaN,0.00,Closed,NaN,NaN,NaN
21745,2019-11-16 00:00:00+00:00,0.42,ETH-A,121.69,51.11,0.00,20.00,0.00,121.69,NaN,...,0.00,0.04,NaN,NaN,NaN,0.00,Closed,NaN,NaN,NaN
21743,2019-11-17 00:00:00+00:00,0.43,ETH-A,123.31,53.02,0.00,20.00,0.00,123.31,NaN,...,0.00,0.04,NaN,NaN,NaN,0.00,Closed,NaN,NaN,NaN
21742,2019-11-18 00:00:00+00:00,"26,313.39",ETH-A,118.39,"3,115,153.99","50,000,000.00",20.00,2.00,177.58,"1,558,617.22",...,37.68,0.04,"62,331.01","1,558,275.09","145,487.20",0.00,Open,3.00,"4,672,730.98","3,504,548.23"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37,2024-03-17 00:00:00+00:00,"444,063.07",ETH-A,"2,504.00","1,111,933,925.89","428,458,674.37","7,500.00",4.51,"3,630.80","246,751,192.26",...,"135,601.75",0.15,"37,629,556.50","246,751,192.26","209,313,743.80","428,458,674.37",Open,6.53,"1,612,304,192.54","1,168,920,539.59"
28,2024-03-18 00:00:00+00:00,"443,694.77",ETH-A,"2,439.83","1,082,537,747.09","428,458,674.37","7,500.00",4.38,"3,537.75","247,201,188.46",...,"77,882.14",0.15,"37,698,180.92","247,201,188.46","206,241,667.90","428,458,674.37",Open,6.35,"1,569,679,733.27","1,138,017,806.62"
18,2024-03-19 00:00:00+00:00,"442,822.50",ETH-A,"2,262.59","1,001,924,077.88","428,458,674.37","7,500.00",4.08,"3,280.75","245,449,240.77",...,"171,181.44",0.15,"37,431,008.90","245,449,240.77","199,379,356.93","428,458,674.37",Open,5.92,"1,452,789,912.92","1,053,272,686.87"
16,2024-03-20 00:00:00+00:00,"444,855.53",ETH-A,"2,402.78","1,068,888,133.11","428,458,674.37","7,500.00",4.35,"3,484.03","245,482,433.74",...,"136,276.39",0.15,"37,436,070.83","245,482,433.74","203,058,946.36","428,458,674.37",Open,6.31,"1,549,887,793.01","1,123,668,649.93"


In [212]:
no_nan_vaults[no_nan_vaults['ilk']=='WSTETH-A'].head()

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,...,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,status,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
9079,2021-10-28 00:00:00+00:00,0.00,WSTETH-A,"2,697.62",0.00,"3,000,000.00","10,000.00",0.00,0.00,0.00,...,0.00,0.04,0.00,0.00,"76,729,520.44",0.00,Open,0.00,0.00,0.00
9075,2021-10-29 00:00:00+00:00,305.63,WSTETH-A,"2,857.24","873,269.68","3,000,000.00","10,000.00",1.38,0.00,"632,462.53",...,40.06,0.04,"25,298.50","632,462.53","80,139,053.59","3,000,000.00",Open,0.00,0.00,0.00
9062,2021-10-30 00:00:00+00:00,674.45,WSTETH-A,"2,811.58","1,896,273.33","4,078,604.29","10,000.00",1.70,0.00,"1,118,390.53",...,128.36,0.04,"44,735.62","1,118,390.53","81,886,606.49","3,000,000.00",Open,0.00,0.00,0.00
9056,2021-10-31 00:00:00+00:00,962.89,WSTETH-A,"2,730.51","2,629,187.21","4,353,719.21","10,000.00",1.92,0.00,"1,368,324.39",...,99.96,0.04,"54,732.98","1,368,324.39","85,368,386.53","4,078,604.29",Open,0.00,0.00,0.00
9043,2021-11-01 00:00:00+00:00,"3,417.46",WSTETH-A,"2,831.66","9,677,082.31","4,443,895.33","10,000.00",2.18,0.00,"4,442,038.79",...,264.64,0.04,"177,681.56","4,442,038.79","84,031,640.03","4,353,719.21",Open,0.00,0.00,0.00


In [213]:
top_10_ilks = top_10_vaults.index.tolist()

In [214]:
top_10_ilks

['ETH-A',
 'WBTC-A',
 'WSTETH-A',
 'ETH-C',
 'ETH-B',
 'WSTETH-B',
 'USDC-A',
 'RWA013-A',
 'WBTC-C',
 'RWA012-A']

In [215]:
topvaults = no_nan_vaults[no_nan_vaults['ilk'].isin(top_10_ilks)]

In [216]:
wbtc_a_vault = no_nan_vaults[no_nan_vaults['ilk']=='WBTC-A']
eth_a_vault = no_nan_vaults[no_nan_vaults['ilk']=='ETH-A']
wsteth_a_vault = no_nan_vaults[no_nan_vaults['ilk']=='WSTETH-A']
eth_c_vault = no_nan_vaults[no_nan_vaults['ilk']=='ETH-C']
eth_b_vault = no_nan_vaults[no_nan_vaults['ilk']=='ETH-B']

In [217]:
#We start with eth-a vault, which has long history and has generated most revenues 
eth_a_vault.head()

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,...,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,status,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
21750,2019-11-13 00:00:00+00:00,0.42,ETH-A,0.00,0.00,0.00,20.00,0.00,0.00,0.00,...,0.00,0.04,0.00,0.00,0.00,0.00,Closed,0.00,0.00,0.00
21747,2019-11-15 00:00:00+00:00,0.42,ETH-A,119.74,50.29,0.00,20.00,0.00,119.74,0.00,...,0.00,0.04,0.00,0.00,0.00,0.00,Closed,0.00,0.00,0.00
21745,2019-11-16 00:00:00+00:00,0.42,ETH-A,121.69,51.11,0.00,20.00,0.00,121.69,0.00,...,0.00,0.04,0.00,0.00,0.00,0.00,Closed,0.00,0.00,0.00
21743,2019-11-17 00:00:00+00:00,0.43,ETH-A,123.31,53.02,0.00,20.00,0.00,123.31,0.00,...,0.00,0.04,0.00,0.00,0.00,0.00,Closed,0.00,0.00,0.00
21742,2019-11-18 00:00:00+00:00,"26,313.39",ETH-A,118.39,"3,115,153.99","50,000,000.00",20.00,2.00,177.58,"1,558,617.22",...,37.68,0.04,"62,331.01","1,558,275.09","145,487.20",0.00,Open,3.00,"4,672,730.98","3,504,548.23"


In [218]:
eth_a_vault.describe()

,cumulative_collateral,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,liquidation_penalty,liquidation_ratio,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
count,"1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00"
mean,"1,458,734.70","1,180.31","1,674,815,093.22","854,497,895.35","7,344.54",2.66,"1,724.99","693,477,623.79",0.07,1.47,"70,427.29",0.03,"21,123,475.17","693,604,670.68","51,565,245.82","854,223,944.79",3.91,"2,461,159,737.81","1,808,848,479.57"
std,"867,527.31",785.15,"1,760,759,531.51","836,076,226.20","5,719.60",0.59,"1,141.89","788,907,642.18",0.16,0.08,"97,692.23",0.02,"27,873,569.54","789,103,240.00","53,855,258.00","836,286,080.97",0.84,"2,584,183,106.69","1,897,827,173.00"
min,0.42,0.00,0.00,0.00,20.00,0.00,0.00,0.00,-1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,"517,750.16",396.93,"522,930,296.41","350,558,569.73",500.00,2.23,595.39,"179,038,619.21",0.00,1.45,"10,818.52",0.01,"3,031,349.40","179,038,619.21","12,664,699.17","350,558,569.73",3.30,"758,614,754.09","550,071,332.69"
50%,"1,733,410.48","1,171.43","673,651,523.57","421,276,896.85","7,500.00",2.56,"1,708.09","216,232,858.05",0.13,1.45,"26,452.76",0.02,"7,708,878.63","216,270,428.76","35,828,999.87","421,276,896.85",3.75,"992,240,131.57","726,560,471.73"
75%,"2,075,231.70","1,624.46","3,121,970,537.42","1,494,643,974.46","15,000.00",3.03,"2,372.86","1,340,082,473.97",0.13,1.50,"108,883.03",0.04,"33,206,894.43","1,340,082,473.97","73,813,092.27","1,494,643,974.46",4.42,"4,612,763,346.09","3,407,265,929.20"
max,"2,966,468.34","3,298.34","7,428,423,394.60","3,395,556,201.44","15,000.00",5.27,"4,782.60","3,077,919,054.99",0.13,1.50,"1,475,897.04",0.15,"142,209,506.83","3,077,919,054.99","351,700,458.61","3,395,556,201.44",7.63,"11,142,635,091.90","8,356,976,318.93"


In [219]:
eth_a_vault.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized stability fee', 'annualized_revenues', 'dart',
       'total_ann_revenues', 'prev_dai_ceiling', 'status',
       'market_collateral_ratio', 'collateral_usd',
       'hypothetical_dai_ceiling'],
      dtype='object')

In [220]:
#7 day MA
eth_a_vault['market_price_7d_ma'] = eth_a_vault['market_price'].rolling(window=7).mean()
eth_a_vault['collateral_usd_7d_ma'] = eth_a_vault['collateral_usd'].rolling(window=7).mean()
eth_a_vault['debt_balance_7d_ma'] = eth_a_vault['debt_balance'].rolling(window=7).mean()
eth_a_vault['safety_collateral_ratio_7d_ma'] = eth_a_vault['safety_collateral_ratio'].rolling(window=7).mean()
eth_a_vault['market_collateral_ratio_7d_ma'] = eth_a_vault['market_collateral_ratio'].rolling(window=7).mean()
eth_a_vault['daily_revenues_7d_ma'] = eth_a_vault['daily_revenues'].rolling(window=7).mean()

# Calculate 30-day moving averages
eth_a_vault['market_price_30d_ma'] = eth_a_vault['market_price'].rolling(window=30).mean()
eth_a_vault['collateral_usd_30d_ma'] = eth_a_vault['collateral_usd'].rolling(window=30).mean()
eth_a_vault['debt_balance_30d_ma'] = eth_a_vault['debt_balance'].rolling(window=30).mean()
eth_a_vault['cumulative_collateral_30d_ma'] = eth_a_vault['cumulative_collateral'].rolling(window=30).mean()
eth_a_vault['safety_collateral_ratio_30d_ma'] = eth_a_vault['safety_collateral_ratio'].rolling(window=30).mean()
eth_a_vault['market_collateral_ratio_30d_ma'] = eth_a_vault['market_collateral_ratio'].rolling(window=30).mean()
eth_a_vault['daily_revenues_30d_ma'] = eth_a_vault['daily_revenues'].rolling(window=30).mean()

#calculate 90 day ma stability fee
eth_a_vault['annualized stability fee_90d_ma'] = eth_a_vault['annualized stability fee'].rolling(window=90).mean()

# Display the head of the DataFrame to verify the new columns
print(eth_a_vault[['market_price', 'market_price_7d_ma', 'market_price_30d_ma', 
                   'collateral_usd', 'collateral_usd_7d_ma', 'collateral_usd_30d_ma',
                   'debt_balance', 'debt_balance_7d_ma', 'debt_balance_30d_ma']].tail())

    market_price  market_price_7d_ma  market_price_30d_ma   collateral_usd  \
37      3,630.80            3,808.92             3,441.50 1,612,304,192.54   
28      3,537.75            3,736.15             3,466.79 1,569,679,733.27   
18      3,280.75            3,642.18             3,481.45 1,452,789,912.92   
16      3,484.03            3,565.54             3,498.75 1,549,887,793.01   
7       3,420.85            3,511.24             3,513.14 1,450,111,134.54   

    collateral_usd_7d_ma  collateral_usd_30d_ma   debt_balance  \
37      1,697,607,736.84       1,510,593,119.23 246,751,192.26   
28      1,661,773,903.88       1,522,773,914.13 247,201,188.46   
18      1,618,456,795.85       1,530,134,155.59 245,449,240.77   
16      1,583,861,447.21       1,538,972,026.37 245,482,433.74   
7       1,549,619,887.15       1,544,225,805.81 216,024,445.89   

    debt_balance_7d_ma  debt_balance_30d_ma  
37      247,744,408.07       226,233,711.18  
28      247,374,304.56       227,487,785.2

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_10768\4234009897.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eth_a_vault['market_price_7d_ma'] = eth_a_vault['market_price'].rolling(window=7).mean()
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_10768\4234009897.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eth_a_vault['collateral_usd_7d_ma'] = eth_a_vault['collateral_usd'].rolling(window=7).mean()
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_10768\4234009897.py:4: SettingWithCopyWarn

In [221]:
# Calculate percent change for selected columns
for column in ['debt_balance', 'cumulative_collateral', 
               'safety_price', 'safety_collateral_ratio', 
               'market_collateral_ratio','annualized stability fee','daily_revenues']:
    eth_a_vault[f'{column}_pct_change'] = eth_a_vault[column].pct_change()

# Calculate volatility (standard deviation) of the percent changes over a 7-day rolling window
for column in ['debt_balance_pct_change', 'cumulative_collateral_pct_change', 
               'safety_price_pct_change', 'safety_collateral_ratio_pct_change', 
               'market_collateral_ratio_pct_change','annualized stability fee_pct_change','daily_revenues_pct_change']:
    eth_a_vault[f'{column}_volatility_7d'] = eth_a_vault[column].rolling(window=7).std()

# Calculate lag for selected columns (1-day lag as an example)
for column in ['debt_balance', 'cumulative_collateral', 
               'safety_price', 'safety_collateral_ratio', 
               'market_collateral_ratio','annualized stability fee','daily_revenues']:
    eth_a_vault[f'{column}_lag30'] = eth_a_vault[column].shift(30)


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_10768\1500168907.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eth_a_vault[f'{column}_pct_change'] = eth_a_vault[column].pct_change()
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_10768\1500168907.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eth_a_vault[f'{column}_pct_change'] = eth_a_vault[column].pct_change()
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_10768\1500168907.py:5: SettingWithCopyWarning: 
A value is trying to be set on a c

In [222]:
eth_a_vault.columns = [f'eth_a_vault_{col}' if col != 'period' and not col.startswith('day') else col for col in eth_a_vault.columns]

In [223]:
eth_a_vault.set_index('day', inplace=True)
eth_a_vault = eth_a_vault.drop(columns=['eth_a_vault_ilk'])

In [ ]:
eth_a_vault['']

In [224]:
eth_a_vault.columns

Index(['eth_a_vault_cumulative_collateral', 'eth_a_vault_safety_price',
       'eth_a_vault_usd_safety_value', 'eth_a_vault_dai_ceiling',
       'eth_a_vault_dai_floor', 'eth_a_vault_safety_collateral_ratio',
       'eth_a_vault_market_price', 'eth_a_vault_debt_balance',
       'eth_a_vault_liquidation_penalty', 'eth_a_vault_liquidation_ratio',
       'eth_a_vault_daily_revenues', 'eth_a_vault_annualized stability fee',
       'eth_a_vault_annualized_revenues', 'eth_a_vault_dart',
       'eth_a_vault_total_ann_revenues', 'eth_a_vault_prev_dai_ceiling',
       'eth_a_vault_status', 'eth_a_vault_market_collateral_ratio',
       'eth_a_vault_collateral_usd', 'eth_a_vault_hypothetical_dai_ceiling',
       'eth_a_vault_market_price_7d_ma', 'eth_a_vault_collateral_usd_7d_ma',
       'eth_a_vault_debt_balance_7d_ma',
       'eth_a_vault_safety_collateral_ratio_7d_ma',
       'eth_a_vault_market_collateral_ratio_7d_ma',
       'eth_a_vault_daily_revenues_7d_ma', 'eth_a_vault_market_price_30d_m

In [225]:
dai_maturity_df_reindexed.head()

,day,dai_maturity_outflow_1-block,dai_maturity_outflow_1-day,dai_maturity_outflow_1-month,dai_maturity_outflow_1-week,dai_maturity_outflow_1-year,dai_maturity_outflow_3-months,dai_maturity_outflow_dai_only_1-block,dai_maturity_outflow_dai_only_1-day,dai_maturity_outflow_dai_only_1-month,dai_maturity_outflow_dai_only_1-week,dai_maturity_outflow_dai_only_1-year,dai_maturity_outflow_dai_only_3-months,dai_maturity_outflow_surplus_buffer_1-block,dai_maturity_outflow_surplus_buffer_1-day,dai_maturity_outflow_surplus_buffer_1-month,dai_maturity_outflow_surplus_buffer_1-week,dai_maturity_outflow_surplus_buffer_1-year,dai_maturity_outflow_surplus_buffer_3-months
0,2019-11-13 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2019-11-14 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2019-11-15 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2019-11-16 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2019-11-17 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [226]:
eth_a_vault = eth_a_vault.merge(dai_maturity_df_reindexed, on=['day'], how='inner')

In [227]:
stable_coin_ratios.head()

,stablecoins_ratio,usdc_ratio
day,,
2024-03-21 00:00:00+00:00,0.11,0.11
2024-03-20 00:00:00+00:00,0.10,0.10
2024-03-19 00:00:00+00:00,0.09,0.09
2024-03-18 00:00:00+00:00,0.08,0.08
2024-03-17 00:00:00+00:00,0.13,0.13


In [228]:
#eth_a_vault = eth_a_vault.merge(stable_coin_ratios, on=['day'], how='inner')

In [229]:
eth_a_vault.head()

,day,eth_a_vault_cumulative_collateral,eth_a_vault_safety_price,eth_a_vault_usd_safety_value,eth_a_vault_dai_ceiling,eth_a_vault_dai_floor,eth_a_vault_safety_collateral_ratio,eth_a_vault_market_price,eth_a_vault_debt_balance,eth_a_vault_liquidation_penalty,...,dai_maturity_outflow_dai_only_1-month,dai_maturity_outflow_dai_only_1-week,dai_maturity_outflow_dai_only_1-year,dai_maturity_outflow_dai_only_3-months,dai_maturity_outflow_surplus_buffer_1-block,dai_maturity_outflow_surplus_buffer_1-day,dai_maturity_outflow_surplus_buffer_1-month,dai_maturity_outflow_surplus_buffer_1-week,dai_maturity_outflow_surplus_buffer_1-year,dai_maturity_outflow_surplus_buffer_3-months
0,2019-11-13 00:00:00+00:00,0.42,0.00,0.00,0.00,20.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2019-11-15 00:00:00+00:00,0.42,119.74,50.29,0.00,20.00,0.00,119.74,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2019-11-16 00:00:00+00:00,0.42,121.69,51.11,0.00,20.00,0.00,121.69,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2019-11-17 00:00:00+00:00,0.43,123.31,53.02,0.00,20.00,0.00,123.31,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2019-11-18 00:00:00+00:00,"26,313.39",118.39,"3,115,153.99","50,000,000.00",20.00,2.00,177.58,"1,558,617.22",0.00,...,"44,042.45","201,006.77","1,173,199.67","64,980.67",0.00,0.00,0.00,0.00,90.79,0.00


In [230]:
start_date = eth_a_vault['day'].min()
end_date = eth_a_vault['day'].max()
date_range = pd.date_range(start=start_date, end=end_date)

psm_full_range_df = pd.DataFrame(index=date_range)



In [231]:
psm_full_range_df.head()

""
2019-11-13 00:00:00+00:00
2019-11-14 00:00:00+00:00
2019-11-15 00:00:00+00:00
2019-11-16 00:00:00+00:00
2019-11-17 00:00:00+00:00


In [232]:


psm_columns = ['psm_change', 'psm_change_excl_rwa', 'psm_change_excl_rwa_30d_avg', 'psm_change_excl_rwa_7d_avg', 'psm_fees', 'psm_inflow', 'psm_inflow_exl_rwa', 'psm_lifetime_fees', 'psm_lifetime_turnover', 'psm_outflow', 'psm_balance', 'psm_turnover']
for column in psm_columns:
    psm_full_range_df[column] = psm_stats_csv[column]

# Fill missing values with zeros
psm_full_range_df.fillna(0, inplace=True)

In [233]:
eth_a_vault.describe()

,eth_a_vault_cumulative_collateral,eth_a_vault_safety_price,eth_a_vault_usd_safety_value,eth_a_vault_dai_ceiling,eth_a_vault_dai_floor,eth_a_vault_safety_collateral_ratio,eth_a_vault_market_price,eth_a_vault_debt_balance,eth_a_vault_liquidation_penalty,eth_a_vault_liquidation_ratio,...,dai_maturity_outflow_dai_only_1-month,dai_maturity_outflow_dai_only_1-week,dai_maturity_outflow_dai_only_1-year,dai_maturity_outflow_dai_only_3-months,dai_maturity_outflow_surplus_buffer_1-block,dai_maturity_outflow_surplus_buffer_1-day,dai_maturity_outflow_surplus_buffer_1-month,dai_maturity_outflow_surplus_buffer_1-week,dai_maturity_outflow_surplus_buffer_1-year,dai_maturity_outflow_surplus_buffer_3-months
count,"1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00",...,"1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00"
mean,"1,459,396.78","1,179.55","1,675,246,787.68","854,770,473.22","7,344.44",2.66,"1,723.91","693,783,096.07",0.07,1.47,...,"121,829,547.87","534,720,838.07","2,643,717,876.30","179,748,513.26",0.00,0.00,0.00,0.00,"43,165,723.38",0.00
std,"867,409.61",784.84,"1,761,240,268.84","836,274,295.12","5,721.43",0.59,"1,141.45","789,067,601.73",0.16,0.08,...,"86,900,656.56","372,490,872.45","1,759,401,866.73","128,214,083.44",0.00,0.00,0.00,0.00,"30,383,977.10",0.00
min,0.42,0.00,0.00,0.00,20.00,0.00,0.00,0.00,-1.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"-5,674,219.65",0.00
25%,"517,894.21",396.45,"522,790,141.46","350,558,569.73",500.00,2.23,594.67,"179,036,518.30",0.00,1.45,...,"42,015,130.91","170,882,210.19","570,286,315.21","61,989,537.40",0.00,0.00,0.00,0.00,"4,001,819.24",0.00
50%,"1,734,175.26","1,171.10","673,540,872.03","421,276,896.85","7,500.00",2.56,"1,707.98","216,249,923.32",0.13,1.45,...,"118,349,364.98","539,666,807.68","3,212,265,572.50","174,613,817.18",0.00,0.00,0.00,0.00,"53,218,504.18",0.00
75%,"2,075,622.02","1,622.45","3,122,870,291.31","1,496,694,216.25","15,000.00",3.03,"2,370.73","1,340,283,261.88",0.13,1.50,...,"168,159,893.31","742,722,843.39","3,705,032,007.03","248,104,760.62",0.00,0.00,0.00,0.00,"69,846,462.07",0.00
max,"2,966,468.34","3,298.34","7,428,423,394.60","3,395,556,201.44","15,000.00",5.27,"4,782.60","3,077,919,054.99",0.13,1.50,...,"314,069,539.85","1,342,922,289.47","5,952,992,110.30","463,381,288.30",0.00,0.00,0.00,0.00,"83,550,327.25",0.00


In [362]:
eth_a_vault['status']

KeyError: 'status'

In [235]:
psm_full_range_df = psm_full_range_df.rename_axis('day')

In [236]:
psm_full_range_df.duplicated().sum()

424

In [237]:
eth_a_vault= eth_a_vault.merge(psm_full_range_df, on=['day'], how='inner')

In [238]:
eth_a_vault[['day','psm_change']].tail()

,day,psm_change
1558,2024-03-16 00:00:00+00:00,"6,819,615.50"
1559,2024-03-17 00:00:00+00:00,"64,591,276.21"
1560,2024-03-18 00:00:00+00:00,"-232,459,681.52"
1561,2024-03-19 00:00:00+00:00,"4,430,852.16"
1562,2024-03-20 00:00:00+00:00,"47,004,882.05"


In [239]:
where_is_dai_csv_table_full = pd.DataFrame(index=date_range)


where_is_dai_csv_table_columns = where_is_dai_csv_table.columns
for column in where_is_dai_csv_table_columns:
    where_is_dai_csv_table_full[column] = where_is_dai_csv_table[column]

# Fill missing values with zeros
where_is_dai_csv_table_full.fillna(0, inplace=True)

In [240]:
where_is_dai_csv_table_full = where_is_dai_csv_table_full.rename_axis('day')

In [241]:
where_is_dai_csv_table_full.head()

,where_is_dai_Bridge,where_is_dai_CeFi,where_is_dai_Dai Savings,where_is_dai_Dex,where_is_dai_EOA,where_is_dai_Lending,where_is_dai_Other,where_is_dai_Treasury
day,,,,,,,,
2019-11-13 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-11-14 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-11-15 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-11-16 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-11-17 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [242]:
eth_a_vault= eth_a_vault.merge(where_is_dai_csv_table_full, on=['day'], how='inner')

In [243]:
eth_a_vault.head()

,day,eth_a_vault_cumulative_collateral,eth_a_vault_safety_price,eth_a_vault_usd_safety_value,eth_a_vault_dai_ceiling,eth_a_vault_dai_floor,eth_a_vault_safety_collateral_ratio,eth_a_vault_market_price,eth_a_vault_debt_balance,eth_a_vault_liquidation_penalty,...,psm_balance,psm_turnover,where_is_dai_Bridge,where_is_dai_CeFi,where_is_dai_Dai Savings,where_is_dai_Dex,where_is_dai_EOA,where_is_dai_Lending,where_is_dai_Other,where_is_dai_Treasury
0,2019-11-13 00:00:00+00:00,0.42,0.00,0.00,0.00,20.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2019-11-15 00:00:00+00:00,0.42,119.74,50.29,0.00,20.00,0.00,119.74,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2019-11-16 00:00:00+00:00,0.42,121.69,51.11,0.00,20.00,0.00,121.69,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2019-11-17 00:00:00+00:00,0.43,123.31,53.02,0.00,20.00,0.00,123.31,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2019-11-18 00:00:00+00:00,"26,313.39",118.39,"3,115,153.99","50,000,000.00",20.00,2.00,177.58,"1,558,617.22",0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [244]:
daily_surplus_buffer_csv = daily_surplus_buffer_csv.fillna(0)

In [245]:
eth_a_vault= eth_a_vault.merge(daily_surplus_buffer_csv, on=['day'], how='inner')

In [363]:
eth_a_vault['eth_a_vault_status']

0       Closed
1       Closed
2       Closed
3       Closed
4         Open
         ...  
1558      Open
1559      Open
1560      Open
1561      Open
1562      Open
Name: eth_a_vault_status, Length: 1563, dtype: object

In [247]:
dsr_df.fillna(0, inplace=True)

In [248]:
dsr_df.head()

,dsr_inflow,dsr_interest,dsr_outflow,dsr_balance,dsr_rate,dai_circulating,dai_total_balance,dai_percent_in_dsr,dai_percent_out_dsr
day,,,,,,,,,
2024-03-27,"5,288,164.43","146,319.97",0.00,"1,545,849,069.62",0.15,"3,198,754,360.76","4,744,603,430.38",0.33,0.67
2024-03-26,"24,182,507.40","570,070.21",0.00,"1,510,137,336.77",0.15,"3,243,870,446.94","4,754,007,783.71",0.32,0.68
2024-03-25,0.00,"591,988.67","79,508,005.52","1,485,384,759.15",0.15,"3,238,774,686.95","4,724,159,446.11",0.31,0.69
2024-03-24,"32,601,790.65","589,190.34",0.00,"1,564,300,776.00",0.15,"3,258,196,717.18","4,822,497,493.18",0.32,0.68
2024-03-23,"42,441,802.19","577,016.24",0.00,"1,531,109,795.00",0.15,"3,245,174,169.16","4,776,283,964.16",0.32,0.68


In [249]:
dsr_df.reset_index(inplace=True)
eth_a_vault.reset_index(inplace=True)
# Remove timezone information from both 'day' columns
eth_a_vault['day'] = eth_a_vault['day'].dt.tz_localize(None)
dsr_df['day'] = dsr_df['day'].dt.tz_localize(None)

In [250]:
dsr_df_full = pd.DataFrame(index=date_range)
dsr_df_full = dsr_df_full.rename_axis('day')

dsr_df_full.reset_index(inplace=True)
dsr_df_full['day'] = dsr_df_full['day'].dt.tz_localize(None)


In [251]:
dsr_df_full = dsr_df_full.merge(dsr_df, on=['day'], how='outer')

In [252]:
dsr_df_full.fillna(0, inplace=True)

In [253]:
dsr_df_full

,day,dsr_inflow,dsr_interest,dsr_outflow,dsr_balance,dsr_rate,dai_circulating,dai_total_balance,dai_percent_in_dsr,dai_percent_out_dsr
0,2019-11-13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2019-11-14,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2019-11-15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2019-11-16,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2019-11-17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
1592,2024-03-25,0.00,"591,988.67","79,508,005.52","1,485,384,759.15",0.15,"3,238,774,686.95","4,724,159,446.11",0.31,0.69
1593,2024-03-24,"32,601,790.65","589,190.34",0.00,"1,564,300,776.00",0.15,"3,258,196,717.18","4,822,497,493.18",0.32,0.68
1594,2024-03-23,"42,441,802.19","577,016.24",0.00,"1,531,109,795.00",0.15,"3,245,174,169.16","4,776,283,964.16",0.32,0.68
1595,2024-03-22,"80,719,471.60","542,875.90",0.00,"1,488,090,976.57",0.15,"3,217,359,661.11","4,705,450,637.68",0.32,0.68


In [254]:
dsr_df_full = dsr_df_full.set_index('day')

In [255]:
#dsr_df_full = dsr_df_full.drop(columns=['level_0','index'])

In [256]:
dsr_df_full.head()

,dsr_inflow,dsr_interest,dsr_outflow,dsr_balance,dsr_rate,dai_circulating,dai_total_balance,dai_percent_in_dsr,dai_percent_out_dsr
day,,,,,,,,,
2019-11-13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-11-14,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-11-15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-11-16,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-11-17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [257]:
eth_a_vault= eth_a_vault.merge(dsr_df_full, on=['day'], how='inner')

In [258]:
eth_a_vault = eth_a_vault.drop(columns=['index'])

In [259]:
eth_a_vault.head()

,day,eth_a_vault_cumulative_collateral,eth_a_vault_safety_price,eth_a_vault_usd_safety_value,eth_a_vault_dai_ceiling,eth_a_vault_dai_floor,eth_a_vault_safety_collateral_ratio,eth_a_vault_market_price,eth_a_vault_debt_balance,eth_a_vault_liquidation_penalty,...,daily_surplus_buffer,dsr_inflow,dsr_interest,dsr_outflow,dsr_balance,dsr_rate,dai_circulating,dai_total_balance,dai_percent_in_dsr,dai_percent_out_dsr
0,2019-11-13,0.42,0.00,0.00,0.00,20.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2019-11-15,0.42,119.74,50.29,0.00,20.00,0.00,119.74,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2019-11-16,0.42,121.69,51.11,0.00,20.00,0.00,121.69,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2019-11-17,0.43,123.31,53.02,0.00,20.00,0.00,123.31,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2019-11-18,"26,313.39",118.39,"3,115,153.99","50,000,000.00",20.00,2.00,177.58,"1,558,617.22",0.00,...,90.78,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [260]:
pivoted_balance_sheet = pivoted_balance_sheet.rename_axis('day')

In [261]:
pivoted_income_statement.rename(columns={'period':'day'}, inplace=True)

In [262]:
pivoted_balance_sheet.reset_index(inplace=True)
pivoted_balance_sheet['day'] = pivoted_balance_sheet['day'].dt.tz_localize(None)

In [263]:
pivoted_balance_sheet = pivoted_balance_sheet.set_index('day')

In [264]:
pivoted_balance_sheet = pivoted_balance_sheet.fillna(0)

In [382]:
print(pivoted_balance_sheet.describe())

       b_s_Crypto-Loans           b_s_DAI           b_s_DSR     b_s_Equity  \
count          1,360.00          1,360.00          1,360.00       1,360.00   
mean   2,290,165,150.68 -4,741,579,818.97   -317,918,673.72 -50,907,047.75   
std    1,259,469,727.42  2,522,050,832.59    522,223,094.31  26,391,917.31   
min      144,805,383.35 -9,951,190,219.65 -1,750,756,886.52 -83,598,090.64   
25%    1,334,072,649.50 -6,326,898,796.77   -258,531,477.03 -70,867,009.35   
50%    2,176,102,429.51 -4,846,370,252.24    -92,004,368.45 -58,130,985.74   
75%    2,843,872,505.76 -3,656,086,113.00    -12,338,379.66 -40,493,992.85   
max    6,135,551,191.72   -108,067,101.21     -1,167,096.23    -200,158.70   

       b_s_Others assets  b_s_Real-World Assets  b_s_Stablecoins  \
count           1,360.00               1,360.00         1,360.00   
mean           30,392.83         580,607,673.57 2,239,602,323.36   
std            23,045.94         880,997,103.86 1,686,843,867.50   
min                 0.00 

In [266]:
eth_a_vault= eth_a_vault.merge(pivoted_balance_sheet, on=['day'], how='left') 

In [267]:
eth_a_vault = eth_a_vault.fillna(0)

In [268]:
eth_a_vault.shape[0]

1563

In [269]:
print(list(eth_a_vault.columns))

['day', 'eth_a_vault_cumulative_collateral', 'eth_a_vault_safety_price', 'eth_a_vault_usd_safety_value', 'eth_a_vault_dai_ceiling', 'eth_a_vault_dai_floor', 'eth_a_vault_safety_collateral_ratio', 'eth_a_vault_market_price', 'eth_a_vault_debt_balance', 'eth_a_vault_liquidation_penalty', 'eth_a_vault_liquidation_ratio', 'eth_a_vault_daily_revenues', 'eth_a_vault_annualized stability fee', 'eth_a_vault_annualized_revenues', 'eth_a_vault_dart', 'eth_a_vault_total_ann_revenues', 'eth_a_vault_prev_dai_ceiling', 'eth_a_vault_status', 'eth_a_vault_market_collateral_ratio', 'eth_a_vault_collateral_usd', 'eth_a_vault_hypothetical_dai_ceiling', 'eth_a_vault_market_price_7d_ma', 'eth_a_vault_collateral_usd_7d_ma', 'eth_a_vault_debt_balance_7d_ma', 'eth_a_vault_safety_collateral_ratio_7d_ma', 'eth_a_vault_market_collateral_ratio_7d_ma', 'eth_a_vault_daily_revenues_7d_ma', 'eth_a_vault_market_price_30d_ma', 'eth_a_vault_collateral_usd_30d_ma', 'eth_a_vault_debt_balance_30d_ma', 'eth_a_vault_cumulati

In [270]:
eth_a_vault = eth_a_vault.sort_values(by='day')
pivoted_income_statement = pivoted_income_statement.sort_values(by='day')

In [271]:
pivoted_income_statement.tail()

item,day,1 - PnL,1.1 - Lending Revenues,1.2 - Liquidations Revenues,1.3 - Trading Revenues,1.4 - Lending Expenses,1.5 - Liquidations Expenses,1.6 - Workforce Expenses,1.9 - Net Income,2 - Assets,...,debt_to_equity_Lag_11,debt_ratio_Lag_11,Total_Revenues_Lag_12,Total_Expenses_Lag_12,Net_Income_Lag_12,profit_margin_Lag_12,ROA_Lag_12,ROE_Lag_12,debt_to_equity_Lag_12,debt_ratio_Lag_12
48,2023-11-01,0.00,"29,666,040.79",455.03,0.00,"-6,376,789.60",0.00,"-2,974,302.25","20,315,403.96",0.00,...,67.65,0.99,"1,169,462.77","-1,941,130.27","-771,667.50",-0.00,-0.00,-0.01,68.99,0.99
49,2023-12-01,0.00,"14,270,261.48",0.00,0.00,"-6,484,200.84",0.00,"-2,658,930.56","5,127,130.08",0.00,...,69.23,0.99,"1,364,508.64","-1,849,641.34","-485,132.70",-0.00,-0.00,-0.01,67.65,0.99
50,2024-01-01,0.00,"20,751,255.62","27,376.30",0.00,"-5,943,924.23",0.00,"-5,669,555.33","9,165,152.35",0.00,...,69.70,0.99,"1,540,089.06","-2,003,227.02","-463,137.96",-0.00,-0.00,-0.01,69.23,0.99
51,2024-02-01,0.00,"29,691,662.12",0.00,0.00,"-4,567,709.04",0.00,"-1,581,115.17","23,542,837.91",0.00,...,72.85,0.99,"1,750,712.42","-2,422,673.84","-671,961.42",-0.00,-0.00,-0.01,69.70,0.99
52,2024-03-01,0.00,"18,130,771.84","4,623.88",0.00,"-7,076,544.94",0.00,"-3,857,879.46","7,200,971.32",0.00,...,68.99,0.99,"2,286,044.04","-4,456,169.85","-2,170,125.82",-0.00,-0.00,-0.03,72.85,0.99


In [272]:
total_vault_data = pd.merge_asof(eth_a_vault, pivoted_income_statement, on='day')

In [273]:
print(list(total_vault_data.columns))

['day', 'eth_a_vault_cumulative_collateral', 'eth_a_vault_safety_price', 'eth_a_vault_usd_safety_value', 'eth_a_vault_dai_ceiling', 'eth_a_vault_dai_floor', 'eth_a_vault_safety_collateral_ratio', 'eth_a_vault_market_price', 'eth_a_vault_debt_balance', 'eth_a_vault_liquidation_penalty', 'eth_a_vault_liquidation_ratio', 'eth_a_vault_daily_revenues', 'eth_a_vault_annualized stability fee', 'eth_a_vault_annualized_revenues', 'eth_a_vault_dart', 'eth_a_vault_total_ann_revenues', 'eth_a_vault_prev_dai_ceiling', 'eth_a_vault_status', 'eth_a_vault_market_collateral_ratio', 'eth_a_vault_collateral_usd', 'eth_a_vault_hypothetical_dai_ceiling', 'eth_a_vault_market_price_7d_ma', 'eth_a_vault_collateral_usd_7d_ma', 'eth_a_vault_debt_balance_7d_ma', 'eth_a_vault_safety_collateral_ratio_7d_ma', 'eth_a_vault_market_collateral_ratio_7d_ma', 'eth_a_vault_daily_revenues_7d_ma', 'eth_a_vault_market_price_30d_ma', 'eth_a_vault_collateral_usd_30d_ma', 'eth_a_vault_debt_balance_30d_ma', 'eth_a_vault_cumulati

### Now for CoinGecko Crypto Market Data

In [274]:
#lets get price feeds for accepted collateral types

ir_csv['ilk'].unique()

array(['ETH-A', 'ETH-B', 'ETH-C', 'WBTC-A', 'WBTC-B', 'WBTC-C',
       'WSTETH-A', 'WSTETH-B', 'RWA002-A', 'RWA013-A', 'DIRECT-SPARK-DAI',
       'RWA014-A', 'RWA005-A', 'RWA012-A', 'RWA015-A', 'RWA007-A',
       'RETH-A', 'RWA003-A', 'GUNIV3DAIUSDC2-A', 'CRVV1ETHSTETH-A',
       'USDC-B', 'LINK-A', 'MATIC-A', 'UNIV2USDCETH-A', 'GNO-A',
       'UNIV2DAIUSDC-A', 'YFI-A', 'RWA004-A', 'GUNIV3DAIUSDC1-A',
       'GUSD-A', 'PAXUSD-A', 'USDC-A', 'DIRECT-AAVEV2-DAI',
       'DIRECT-COMPV2-DAI', 'RWA008-A', 'RENBTC-A', 'MANA-A', 'RWA009-A',
       'RWA001-A', 'UNI-A', 'UNIV2DAIETH-A', 'UNIV2WBTCETH-A',
       'UNIV2WBTCDAI-A', 'RWA-001', 'UNIV2UNIETH-A', 'TUSD-A', 'USDP-A',
       'BAT-A', 'BAL-A', 'ZRX-A', 'COMP-A', 'PSM-GUSD-A', 'AAVE-A',
       'UNIV2LINKETH-A', 'KNC-A', 'LRC-A', 'PSM-USDC-A', 'UNIV2AAVEETH-A',
       'UNIV2DAIUSDT-A', 'UNIV2ETHUSDT-A', 'USDT-A', 'PSM-PAX-A',
       'RWA006-A', 'PAX-A', nan, 'USDC', 'SAI'], dtype=object)

In [275]:
#need to use yfinance instead, coingecko clocked to 1 year historical




In [276]:
sp = yf.Ticker("^GSPC")

In [277]:
sp_from_nov = sp.history(period="41mo")

In [278]:
sp_from_nov = sp_from_nov.drop(columns=['Dividends','Stock Splits','Open','Low','High'])

In [279]:
sp_from_nov.columns = [f's&p_500_market_{col}' if col != 'Date' else col for col in sp_from_nov.columns]

In [280]:
sp_from_nov

,s&p_500_market_Close,s&p_500_market_Volume
Date,,
2020-11-04 00:00:00-05:00,"3,443.44",4790400000
2020-11-05 00:00:00-05:00,"3,510.45",4858150000
2020-11-06 00:00:00-05:00,"3,509.44",4842460000
2020-11-09 00:00:00-05:00,"3,550.50",8570510000
2020-11-10 00:00:00-05:00,"3,545.53",6037470000
...,...,...
2024-03-27 00:00:00-04:00,"5,248.49",3850500000
2024-03-28 00:00:00-04:00,"5,254.35",3998270000
2024-04-01 00:00:00-04:00,"5,243.77",3325930000


In [281]:
btc = yf.Ticker('BTC-USD')

In [282]:
btc_from_nov = btc.history(period='41mo')
btc_from_nov = btc_from_nov.drop(columns=['Dividends','Stock Splits','Open','Low','High'])

In [283]:
btc_from_nov.columns = [f'btc_market_{col}' if col != 'Date' else col for col in btc_from_nov.columns]

In [448]:
btc_from_nov.describe()

,btc_market_Close,btc_market_Volume
count,"1,248.00","1,248.00"
mean,"35,553.30","32,292,779,698.55"
std,"13,662.83","20,048,116,628.58"
min,"13,950.30","5,331,172,801.00"
25%,"23,947.60","19,294,405,680.75"
50%,"33,548.94","28,575,169,162.50"
75%,"44,350.21","39,321,396,767.00"
max,"73,083.50","350,967,941,479.00"


In [284]:
eth = yf.Ticker('ETH-USD')

In [285]:
eth_from_nov = eth.history(period='41mo')
eth_from_nov = eth_from_nov.drop(columns=['Dividends','Stock Splits','Open','Low','High'])

In [286]:
eth_from_nov.columns = [f'eth_market_{col}' if col != 'Date' else col for col in eth_from_nov.columns]

In [447]:
eth_from_nov.describe()

,eth_market_Close,eth_market_Volume
count,"1,248.00","1,248.00"
mean,"2,166.39","16,651,333,920.49"
std,920.07,"10,978,619,745.88"
min,387.60,"2,081,625,742.00"
25%,"1,583.44","8,624,262,973.75"
50%,"1,880.81","14,662,045,135.00"
75%,"2,763.88","21,285,832,540.25"
max,"4,812.09","84,482,912,776.00"


In [288]:
mkr = yf.Ticker('MKR-USD')

In [289]:
mkr_from_nov = mkr.history(period='41mo')
mkr_from_nov = mkr_from_nov.drop(columns=['Dividends','Stock Splits','Open','Low','High'])

In [290]:
mkr_from_nov.columns = [f'mkr_market_{col}' if col != 'Date' else col for col in mkr_from_nov.columns]

In [446]:
mkr_from_nov.describe()

,mkr_market_Close,mkr_market_Volume
count,"1,248.00","1,248.00"
mean,"1,663.97","142,485,960.73"
std,993.41,"535,871,504.52"
min,502.06,"10,290,023.00"
25%,775.08,"46,659,678.75"
50%,"1,407.76","80,113,085.00"
75%,"2,257.46","136,346,638.50"
max,"6,012.46","13,754,958,457.00"


In [292]:
dai = yf.Ticker('DAI-USD')

In [293]:
dai_from_nov = dai.history(period='41mo')
dai_from_nov = dai_from_nov.drop(columns=['Dividends','Stock Splits','Open','Low','High'])

In [294]:
dai_from_nov.columns = [f'dai_market_{col}' if col != 'Date' else col for col in dai_from_nov.columns]

In [445]:
dai_from_nov.describe()

,dai_market_Close,dai_market_Volume,deviation,abs_deviation
count,"1,248.00","1,248.00","1,248.00","1,248.00"
mean,1.00,"339,442,488.21",0.00,0.00
std,0.00,"287,632,250.25",0.00,0.00
min,0.97,"45,847,952.00",-0.03,0.00
25%,1.00,"160,987,684.50",-0.00,0.00
50%,1.00,"285,155,008.00",0.00,0.00
75%,1.00,"420,790,948.00",0.00,0.00
max,1.01,"4,642,451,631.00",0.01,0.03


In [440]:
dai_from_nov['deviation'] = dai_from_nov['dai_market_Close'] - 1
dai_from_nov['abs_deviation'] = dai_from_nov['deviation'].abs()

average_deviation = dai_from_nov['abs_deviation'].mean()
standard_deviation = dai_from_nov['dai_market_Close'].std()

print(f"Average Deviation from $1: {average_deviation}")
print(f"Standard Deviation of DAI Price: {standard_deviation}")

Average Deviation from $1: 0.0008210278569887846
Standard Deviation of DAI Price: 0.001636980960230382


In [296]:
crypto_market_data = pd.merge(dai_from_nov, eth_from_nov, on=['Date'], how='inner')

In [297]:
crypto_market_data = crypto_market_data.merge(btc_from_nov, on=['Date'], how='inner')

In [298]:
crypto_market_data = crypto_market_data.merge(mkr_from_nov, on=['Date'], how='inner')

In [299]:
sp_from_nov.index = sp_from_nov.index.tz_convert ('UTC')

In [424]:
sp_from_nov.describe()

,s&p_500_market_Close,s&p_500_market_Volume
count,857.00,857.00
mean,"4,250.12","4,345,673,150.53"
std,365.86,"923,082,409.97"
min,"3,443.44","1,639,500,000.00"
25%,"3,962.71","3,812,470,000.00"
50%,"4,224.79","4,151,200,000.00"
75%,"4,487.46","4,687,320,000.00"
max,"5,254.35","9,976,520,000.00"


In [420]:
print(sp_from_nov.describe())

       s&p_500_market_Close  s&p_500_market_Volume
count                857.00                 857.00
mean               4,250.12       4,345,673,150.53
std                  365.86         923,082,409.97
min                3,443.44       1,639,500,000.00
25%                3,962.71       3,812,470,000.00
50%                4,224.79       4,151,200,000.00
75%                4,487.46       4,687,320,000.00
max                5,254.35       9,976,520,000.00


In [301]:
crypto_market_data.index

DatetimeIndex(['2020-11-03 00:00:00+00:00', '2020-11-04 00:00:00+00:00',
               '2020-11-05 00:00:00+00:00', '2020-11-06 00:00:00+00:00',
               '2020-11-07 00:00:00+00:00', '2020-11-08 00:00:00+00:00',
               '2020-11-09 00:00:00+00:00', '2020-11-10 00:00:00+00:00',
               '2020-11-11 00:00:00+00:00', '2020-11-12 00:00:00+00:00',
               ...
               '2024-03-25 00:00:00+00:00', '2024-03-26 00:00:00+00:00',
               '2024-03-27 00:00:00+00:00', '2024-03-28 00:00:00+00:00',
               '2024-03-29 00:00:00+00:00', '2024-03-30 00:00:00+00:00',
               '2024-03-31 00:00:00+00:00', '2024-04-01 00:00:00+00:00',
               '2024-04-02 00:00:00+00:00', '2024-04-03 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Date', length=1248, freq=None)

In [303]:
sp_from_nov_normalized = sp_from_nov.index.normalize()
crypto_market_data_normalized = crypto_market_data.index.normalize()

In [304]:
sp_from_nov.index = sp_from_nov_normalized
crypto_market_data.index = crypto_market_data_normalized

In [425]:
sp_from_nov.describe()

,s&p_500_market_Close,s&p_500_market_Volume
count,857.00,857.00
mean,"4,250.12","4,345,673,150.53"
std,365.86,"923,082,409.97"
min,"3,443.44","1,639,500,000.00"
25%,"3,962.71","3,812,470,000.00"
50%,"4,224.79","4,151,200,000.00"
75%,"4,487.46","4,687,320,000.00"
max,"5,254.35","9,976,520,000.00"


In [305]:
crypto_market_data = crypto_market_data.merge(sp_from_nov, on=['Date'], how='inner')

In [307]:
for col in crypto_market_data:
    crypto_market_data[f'{col}_daily_returns'] = crypto_market_data[col].pct_change()
    crypto_market_data[f'{col}_7d_ma']= crypto_market_data[col].rolling(window=7).mean()
    crypto_market_data[f'{col}_30d_ma'] = crypto_market_data[col].rolling(window=30).mean()



In [308]:
crypto_market_data.rename_axis('day',inplace=True)

In [309]:
crypto_market_data = crypto_market_data.fillna(0)

In [310]:
nan_rows = crypto_market_data[crypto_market_data.isna().any(axis=1)]
print(nan_rows)


Empty DataFrame
Columns: [dai_market_Close, dai_market_Volume, eth_market_Close, eth_market_Volume, btc_market_Close, btc_market_Volume, mkr_market_Close, mkr_market_Volume, s&p_500_market_Close, s&p_500_market_Volume, dai_market_Close_daily_returns, dai_market_Close_7d_ma, dai_market_Close_30d_ma, dai_market_Volume_daily_returns, dai_market_Volume_7d_ma, dai_market_Volume_30d_ma, eth_market_Close_daily_returns, eth_market_Close_7d_ma, eth_market_Close_30d_ma, eth_market_Volume_daily_returns, eth_market_Volume_7d_ma, eth_market_Volume_30d_ma, btc_market_Close_daily_returns, btc_market_Close_7d_ma, btc_market_Close_30d_ma, btc_market_Volume_daily_returns, btc_market_Volume_7d_ma, btc_market_Volume_30d_ma, mkr_market_Close_daily_returns, mkr_market_Close_7d_ma, mkr_market_Close_30d_ma, mkr_market_Volume_daily_returns, mkr_market_Volume_7d_ma, mkr_market_Volume_30d_ma, s&p_500_market_Close_daily_returns, s&p_500_market_Close_7d_ma, s&p_500_market_Close_30d_ma, s&p_500_market_Volume_daily_

In [311]:
nan_rows.columns

Index(['dai_market_Close', 'dai_market_Volume', 'eth_market_Close',
       'eth_market_Volume', 'btc_market_Close', 'btc_market_Volume',
       'mkr_market_Close', 'mkr_market_Volume', 's&p_500_market_Close',
       's&p_500_market_Volume', 'dai_market_Close_daily_returns',
       'dai_market_Close_7d_ma', 'dai_market_Close_30d_ma',
       'dai_market_Volume_daily_returns', 'dai_market_Volume_7d_ma',
       'dai_market_Volume_30d_ma', 'eth_market_Close_daily_returns',
       'eth_market_Close_7d_ma', 'eth_market_Close_30d_ma',
       'eth_market_Volume_daily_returns', 'eth_market_Volume_7d_ma',
       'eth_market_Volume_30d_ma', 'btc_market_Close_daily_returns',
       'btc_market_Close_7d_ma', 'btc_market_Close_30d_ma',
       'btc_market_Volume_daily_returns', 'btc_market_Volume_7d_ma',
       'btc_market_Volume_30d_ma', 'mkr_market_Close_daily_returns',
       'mkr_market_Close_7d_ma', 'mkr_market_Close_30d_ma',
       'mkr_market_Volume_daily_returns', 'mkr_market_Volume_7d_ma',
  

for column in crypto_market_data[f'{col}_daily_returns']:
    crypto_market_data[f'{column}_volatility_7d'] = eth_a_vault[column].rolling(window=7).std()



# Calculate lag for selected columns (1-day lag as an example)
for column in ['debt_balance', 'cumulative_collateral', 
               'safety_price', 'safety_collateral_ratio', 
               'market_collateral_ratio','annualized stability fee','daily_revenues']:
    eth_a_vault[f'{column}_lag30'] = eth_a_vault[column].shift(30)

## Now for Macro Economic Data from FRED Api

In [312]:
def fetch_and_process_tbill_data(api_url, data_key, date_column, value_column, date_format='datetime'):
    # Retrieve the API key from Streamlit secrets
    api_key = st.secrets["FRED_API_KEY"]

    # Append the API key to the URL
    api_url_with_key = f"{api_url}&api_key={api_key}"

    response = requests.get(api_url_with_key)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data[data_key])
        
        if date_format == 'datetime':
            df[date_column] = pd.to_datetime(df[date_column])
        
        df.set_index(date_column, inplace=True)
        df[value_column] = df[value_column].astype(float)
        return df
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame in case of failure

In [313]:
three_month_tbill_historical_api = "https://api.stlouisfed.org/fred/series/observations?series_id=TB3MS&file_type=json"
three_month_tbill = fetch_and_process_tbill_data(three_month_tbill_historical_api, "observations", "date", "value")


In [314]:
three_month_tbill = three_month_tbill[three_month_tbill.index >= '2019-11-01']

In [315]:
three_month_tbill['3m_tbill'] = three_month_tbill['value'] / 100

In [316]:
three_month_tbill

,realtime_start,realtime_end,value,3m_tbill
date,,,,
2019-11-01,2024-04-03,2024-04-03,1.54,0.02
2019-12-01,2024-04-03,2024-04-03,1.54,0.02
2020-01-01,2024-04-03,2024-04-03,1.52,0.02
2020-02-01,2024-04-03,2024-04-03,1.52,0.02
2020-03-01,2024-04-03,2024-04-03,0.29,0.00
2020-04-01,2024-04-03,2024-04-03,0.14,0.00
2020-05-01,2024-04-03,2024-04-03,0.13,0.00
2020-06-01,2024-04-03,2024-04-03,0.16,0.00
2020-07-01,2024-04-03,2024-04-03,0.13,0.00


In [317]:
tmo_path = '../data/csv/3mo_tbill.csv'
three_month_tbill.to_csv(tmo_path)
three_month_tbill_csv = pd.read_csv(tmo_path, index_col='date', parse_dates=True)

In [318]:
three_month_tbill_csv.describe()

,realtime_start,realtime_end,value,3m_tbill
date,,,,
2023-11-01,2024-04-03,2024-04-03,5.27,0.05
2023-12-01,2024-04-03,2024-04-03,5.24,0.05
2024-01-01,2024-04-03,2024-04-03,5.22,0.05
2024-02-01,2024-04-03,2024-04-03,5.24,0.05
2024-03-01,2024-04-03,2024-04-03,5.24,0.05


In [320]:
#sticky_price_consumer_index

spi_path = "../data/csv/sticky_price_consumer_price_index.csv"
sticky_index = pd.read_csv(spi_path)

In [380]:
print(sticky_index.describe())

       sticky_cpi
count       52.00
mean         3.86
std          1.72
min          1.40
25%          2.31
50%          3.71
75%          5.40
max          6.54


In [321]:
eff_rate_path =  "../data/csv/effective_federal_funds_rate.csv"
fed_funds_rate = pd.read_csv(eff_rate_path)

In [378]:
fed_funds_rate.describe()

,date,effective_funds_rate
count,1152,1152
unique,NaN,34
top,NaN,5.33
freq,NaN,171
mean,2022-01-15 12:00:00+00:00,NaN
min,2019-11-01 00:00:00+00:00,NaN
25%,2020-12-08 18:00:00+00:00,NaN
50%,2022-01-15 12:00:00+00:00,NaN
75%,2023-02-22 06:00:00+00:00,NaN
max,2024-04-01 00:00:00+00:00,NaN


In [322]:
fed_funds_rate=fed_funds_rate.rename(columns={'DATE':'date'})
fed_funds_rate['date'] = pd.to_datetime(fed_funds_rate['date'])

In [323]:
fed_funds_rate['date'] = fed_funds_rate['date'].dt.tz_localize('UTC')

In [377]:
fed_funds_rate.describe()

,date,effective_funds_rate
count,1152,1152
unique,NaN,34
top,NaN,5.33
freq,NaN,171
mean,2022-01-15 12:00:00+00:00,NaN
min,2019-11-01 00:00:00+00:00,NaN
25%,2020-12-08 18:00:00+00:00,NaN
50%,2022-01-15 12:00:00+00:00,NaN
75%,2023-02-22 06:00:00+00:00,NaN
max,2024-04-01 00:00:00+00:00,NaN


In [327]:
three_month_tbill_csv.index = three_month_tbill_csv.index.tz_localize('UTC')

In [329]:
three_month_tbill_csv.tail()

,3_m_tbill_yield,3m_tbill
date,,
2023-11-01 00:00:00+00:00,5.27,0.05
2023-12-01 00:00:00+00:00,5.24,0.05
2024-01-01 00:00:00+00:00,5.22,0.05
2024-02-01 00:00:00+00:00,5.24,0.05
2024-03-01 00:00:00+00:00,5.24,0.05


In [330]:
three_month_tbill_csv = three_month_tbill_csv.rename(columns={'value':'3_m_tbill_yield'})

In [381]:
print(three_month_tbill_csv.describe())

       3_m_tbill_yield  3m_tbill
count            53.00     53.00
mean              2.05      0.02
std               2.20      0.02
min               0.02      0.00
25%               0.09      0.00
50%               0.98      0.01
75%               4.65      0.05
max               5.34      0.05


In [332]:
fed_funds_rate = fed_funds_rate.rename(columns={'EFFR':'effective_funds_rate'})

In [333]:
macro_data = pd.merge(fed_funds_rate,three_month_tbill_csv, on=['date'],how='inner' )

In [334]:
macro_data['effective_funds_rate'] = macro_data['effective_funds_rate'].replace('.', np.nan)

# Convert the column to a numeric type (float), necessary for ffill to work after replacing '.' with np.nan
macro_data['effective_funds_rate'] = pd.to_numeric(macro_data['effective_funds_rate'], errors='coerce')

# Now apply front filling
macro_data['effective_funds_rate'].ffill(inplace=True)

# Display the results


0    1.57
1    1.57
2    0.06
3    0.05
4    0.05
5    0.08
6    0.09
7    0.09
8    0.09
9    0.09
10   0.08
11   0.07
12   0.07
13   0.06
14   0.10
15   0.08
16   0.08
17   0.08
18   0.08
19   0.08
20   0.08
21   0.33
22   0.83
23   1.58
24   2.33
25   2.33
26   3.08
27   3.83
28   4.33
29   4.58
30   4.83
31   5.08
32   5.33
33   5.33
34   5.33
35   5.33
36   5.33
37   5.33
38   5.33
Name: effective_funds_rate, dtype: float64


In [379]:
print(macro_data['effective_funds_rate'].describe())

count   38.00
mean     1.83
std      2.17
min      0.05
25%      0.08
50%      0.22
75%      4.21
max      5.33
Name: effective_funds_rate, dtype: float64


In [335]:
sticky_index.rename(columns={'DATE': 'date', 'CORESTICKM159SFRBATL': 'sticky_cpi'}, inplace=True)

In [336]:
sticky_index.tail()

,date,sticky_cpi
47,2023-10-01,4.88
48,2023-11-01,4.69
49,2023-12-01,4.55
50,2024-01-01,4.60
51,2024-02-01,4.40


In [337]:
sticky_index['date']= pd.to_datetime(sticky_index['date'])

In [338]:
sticky_index['date'] = sticky_index['date'].dt.tz_localize('UTC')

In [339]:
macro_data = macro_data.merge(sticky_index, on=['date'],how='inner')

In [340]:
macro_data = macro_data.drop(columns=['3m_tbill'])

In [341]:
macro_data.set_index('date',inplace=True)

In [342]:
macro_data.rename_axis('day',inplace=True)

In [343]:
macro_data

,effective_funds_rate,3_m_tbill_yield,sticky_cpi
day,,,
2019-11-01 00:00:00+00:00,1.57,1.54,2.77
2020-01-01 00:00:00+00:00,1.57,1.52,2.79
2020-04-01 00:00:00+00:00,0.06,0.14,2.18
2020-05-01 00:00:00+00:00,0.05,0.13,2.01
2020-06-01 00:00:00+00:00,0.05,0.16,1.99
2020-07-01 00:00:00+00:00,0.08,0.13,2.28
2020-09-01 00:00:00+00:00,0.09,0.11,1.98
2020-10-01 00:00:00+00:00,0.09,0.10,1.75
2020-12-01 00:00:00+00:00,0.09,0.09,1.60


In [344]:
crypto_market_data = crypto_market_data.sort_values(by='day')
macro_data = macro_data.sort_values(by='day')



In [345]:
macro_data.tail()

,effective_funds_rate,3_m_tbill_yield,sticky_cpi
day,,,
2023-09-01 00:00:00+00:00,5.33,5.32,4.96
2023-11-01 00:00:00+00:00,5.33,5.27,4.69
2023-12-01 00:00:00+00:00,5.33,5.24,4.55
2024-01-01 00:00:00+00:00,5.33,5.22,4.60
2024-02-01 00:00:00+00:00,5.33,5.24,4.40


In [346]:
macro_and_crypto = pd.merge_asof(crypto_market_data, macro_data, on='day')


In [347]:
macro_and_crypto.shape[0]

857

In [348]:
macro_and_crypto['day'] = macro_and_crypto['day'].dt.tz_convert('UTC')

In [350]:
total_vault_data['day'] = total_vault_data['day'].dt.tz_localize('UTC')

In [351]:
dataset = total_vault_data.merge(macro_and_crypto, on=['day'],how='inner')

In [352]:
nan_rows = dataset[dataset.isna().any(axis=1)]
print(nan_rows)


                          day  eth_a_vault_cumulative_collateral  \
0   2020-11-04 00:00:00+00:00                       2,558,593.17   
1   2020-11-05 00:00:00+00:00                       2,557,079.27   
2   2020-11-06 00:00:00+00:00                       2,557,674.49   
3   2020-11-09 00:00:00+00:00                       2,448,558.85   
4   2020-11-10 00:00:00+00:00                       2,434,900.11   
..                        ...                                ...   
349 2022-03-25 00:00:00+00:00                       1,764,503.58   
350 2022-03-28 00:00:00+00:00                       1,763,916.77   
351 2022-03-29 00:00:00+00:00                       1,765,298.96   
352 2022-03-30 00:00:00+00:00                       1,762,742.40   
353 2022-03-31 00:00:00+00:00                       1,412,615.00   

     eth_a_vault_safety_price  eth_a_vault_usd_safety_value  \
0                      268.33                686,538,775.41   
1                      275.63                704,816,281.

In [353]:
# Checking for columns with NaN values and their count of NaNs
nan_columns = dataset.isna().sum()
print(nan_columns[nan_columns > 0])


Total Revenues_rolling_avg_pct_chg_lag_10       18
Total Revenues_volatility_pct_chg_lag_10        18
Total Revenues_rolling_avg_pct_chg_lag_11       40
Total Revenues_volatility_pct_chg_lag_11        40
Total Revenues_rolling_avg_pct_chg_lag_12       59
Total Revenues_volatility_pct_chg_lag_12        59
Total Expenses_pct_chg                          40
Total Expenses_rolling_avg_pct_chg             101
Total Expenses_volatility_pct_chg              101
Total Expenses_rolling_avg_pct_chg_lag_1       122
Total Expenses_volatility_pct_chg_lag_1        122
Total Expenses_rolling_avg_pct_chg_lag_2       142
Total Expenses_volatility_pct_chg_lag_2        142
Total Expenses_rolling_avg_pct_chg_lag_3       164
Total Expenses_volatility_pct_chg_lag_3        164
Total Expenses_rolling_avg_pct_chg_lag_4       185
Total Expenses_volatility_pct_chg_lag_4        185
Total Expenses_rolling_avg_pct_chg_lag_5       207
Total Expenses_volatility_pct_chg_lag_5        207
Total Expenses_rolling_avg_pct_

In [354]:
dataset_no_nan = dataset.fillna(0)

In [355]:
dataset_no_nan.isna().any().sum()

0

In [356]:
print(list(dataset_no_nan['dai_maturity_outflow_surplus_buffer_1-block']))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [357]:
print(list(dataset_no_nan.columns))

['day', 'eth_a_vault_cumulative_collateral', 'eth_a_vault_safety_price', 'eth_a_vault_usd_safety_value', 'eth_a_vault_dai_ceiling', 'eth_a_vault_dai_floor', 'eth_a_vault_safety_collateral_ratio', 'eth_a_vault_market_price', 'eth_a_vault_debt_balance', 'eth_a_vault_liquidation_penalty', 'eth_a_vault_liquidation_ratio', 'eth_a_vault_daily_revenues', 'eth_a_vault_annualized stability fee', 'eth_a_vault_annualized_revenues', 'eth_a_vault_dart', 'eth_a_vault_total_ann_revenues', 'eth_a_vault_prev_dai_ceiling', 'eth_a_vault_status', 'eth_a_vault_market_collateral_ratio', 'eth_a_vault_collateral_usd', 'eth_a_vault_hypothetical_dai_ceiling', 'eth_a_vault_market_price_7d_ma', 'eth_a_vault_collateral_usd_7d_ma', 'eth_a_vault_debt_balance_7d_ma', 'eth_a_vault_safety_collateral_ratio_7d_ma', 'eth_a_vault_market_collateral_ratio_7d_ma', 'eth_a_vault_daily_revenues_7d_ma', 'eth_a_vault_market_price_30d_ma', 'eth_a_vault_collateral_usd_30d_ma', 'eth_a_vault_debt_balance_30d_ma', 'eth_a_vault_cumulati

In [364]:
dataset_no_nan['eth_a_vault_status']

0      Open
1      Open
2      Open
3      Open
4      Open
       ... 
831    Open
832    Open
833    Open
834    Open
835    Open
Name: eth_a_vault_status, Length: 836, dtype: object

## Running Correlations for Feature Engineering

In [365]:
numeric_dataset = dataset_no_nan.select_dtypes(include=[np.number])
correlations = numeric_dataset.corr()


In [367]:
# Assuming 'df' is your DataFrame
non_numeric_columns = dataset_no_nan.select_dtypes(exclude=[np.number])

# This will show you the columns that do not contain numeric data
print(non_numeric_columns.columns)


Index(['day', 'eth_a_vault_status'], dtype='object')


In [369]:
correlations = numeric_dataset.corr()

collateral_target_correlations = correlations['eth_a_vault_cumulative_collateral'].drop('eth_a_vault_cumulative_collateral')
collateral_sorted_correlations = collateral_target_correlations.sort_values(ascending=False)

# Display the sorted correlations
print(collateral_sorted_correlations)

eth_a_vault_cumulative_collateral_30d_ma   0.99
eth_a_vault_cumulative_collateral_lag30    0.98
debt_ratio_Lag_2                           0.87
debt_ratio_Lag_1                           0.87
debt_ratio_Lag_3                           0.86
                                           ... 
1 - PnL                                     NaN
2 - Assets                                  NaN
2.8 - Operating Reserves                    NaN
3 - Liabilities & Equity                    NaN
3.8 - Equity (Operating Reserves)           NaN
Name: eth_a_vault_cumulative_collateral, Length: 393, dtype: float64


In [370]:
for index, value in collateral_sorted_correlations.items():
    print(f"{index:50} {value}")

eth_a_vault_cumulative_collateral_30d_ma           0.9943398210444682
eth_a_vault_cumulative_collateral_lag30            0.9805547497547766
debt_ratio_Lag_2                                   0.8736438996467315
debt_ratio_Lag_1                                   0.8707493279881601
debt_ratio_Lag_3                                   0.8639384287062543
cumulative_expenses                                0.8509625923156909
debt_ratio_Lag_4                                   0.8473905860144817
debt_ratio_Lag_12                                  0.8427451211411413
debt_ratio                                         0.8422797707986605
debt_ratio_Lag_7                                   0.8396211743519353
debt_ratio_Lag_8                                   0.8394948356457375
debt_ratio_Lag_11                                  0.8386780413470225
debt_ratio_Lag_9                                   0.8364084137718227
debt_ratio_Lag_10                                  0.8356662609559448
debt_ratio_Lag_6    

In [432]:

# Select correlations of all features with the target variable, excluding the target variable itself
etha_target_correlations = correlations['eth_a_vault_liquidation_ratio'].drop('eth_a_vault_liquidation_ratio')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
etha_sorted_correlations = etha_target_correlations.sort_values(ascending=False)

for index, value in etha_sorted_correlations.items():
    print(f"{index:50} {value}")

ROE_Lag_2                                          0.8564102814077655
ROE_Lag_3                                          0.8421159335778359
b_s_Equity_rolling_avg                             0.8300568243606087
b_s_Equity                                         0.8271204896706157
ROE_Lag_5                                          0.7953387104636003
debt_ratio_Lag_6                                   0.7942735290480126
debt_ratio_Lag_7                                   0.7884222499276735
ROE_Lag_4                                          0.7796483453861516
ROE_Lag_1                                          0.7774006733018913
debt_ratio_Lag_5                                   0.7773651106695469
ROE_Lag_6                                          0.7767961913616538
debt_ratio_Lag_8                                   0.7725618759404949
eth_a_vault_cumulative_collateral_30d_ma           0.7615768361141249
eth_a_vault_cumulative_collateral_lag30            0.7596304842848594
eth_a_vault_cumulati

In [371]:
# Select correlations of all features with the target variable, excluding the target variable itself
dsr_target_correlations = correlations['dsr_rate'].drop('dsr_rate')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
dsr_sorted_correlations = dsr_target_correlations.sort_values(ascending=False)

for index, value in dsr_sorted_correlations.items():
    print(f"{index:50} {value}")

dsr_interest                                       0.957173341735613
dai_percent_in_dsr                                 0.8861496852741225
dsr_balance                                        0.8675209608131664
where_is_dai_Dai Savings                           0.8547481095819917
b_s_Real-World_Assets_rolling_avg                  0.806432549779197
b_s_Real-World Assets                              0.8001473169042703
psm_lifetime_turnover                              0.7607235766295427
eth_a_vault_annualized stability fee               0.7106474267890815
effective_funds_rate                               0.7010858581105622
cumulative_revenues                                0.6925449520987872
1.1 - Lending Revenues                             0.6774585340586271
3_m_tbill_yield                                    0.6496205442713194
cumulative_net_income                              0.5571337307824817
s&p_500_market_Close                               0.5546886211827123
dai_total_balance     

In [372]:
# Select correlations of all features with the target variable, excluding the target variable itself
stability_fee_target_correlations = correlations['eth_a_vault_annualized stability fee'].drop('eth_a_vault_annualized stability fee')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
stability_fee_target_correlations = stability_fee_target_correlations.sort_values(ascending=False)

for index, value in stability_fee_target_correlations.items():
    print(f"{index:50} {value}")

eth_a_vault_annualized stability fee_lag30         0.8263602331280309
eth_a_vault_annualized stability fee_90d_ma        0.8064729805305909
dsr_interest                                       0.7126069357733581
dsr_rate                                           0.7106474267890815
1.1 - Lending Revenues                             0.6927947297890712
ROA_Lag_1                                          0.661023675355134
profit_margin_Lag_1                                0.661023675355134
Total_Revenues_Lag_1                               0.6363416597811982
Total Revenues                                     0.6353741235591359
Net_Income_Lag_1                                   0.6281906347395354
ROA                                                0.6249489719613421
profit_margin                                      0.6249489719613421
1.9 - Net Income                                   0.594461311734676
dai_percent_in_dsr                                 0.5934799655401125
eth_a_vault_total_ann_r

In [373]:
# Select correlations of all features with the target variable, excluding the target variable itself
safety_ratio_target_correlations = correlations['eth_a_vault_safety_collateral_ratio'].drop('eth_a_vault_safety_collateral_ratio')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
safety_ratio_target_correlations = safety_ratio_target_correlations.sort_values(ascending=False)

for index, value in safety_ratio_target_correlations.items():
    print(f"{index:50} {value}")

eth_a_vault_market_collateral_ratio                0.9971050651432883
eth_a_vault_safety_collateral_ratio_7d_ma          0.962860916302392
eth_a_vault_market_collateral_ratio_7d_ma          0.9606967723774925
eth_a_vault_safety_collateral_ratio_30d_ma         0.8729676723358981
eth_a_vault_market_collateral_ratio_30d_ma         0.8708217361979157
eth_a_vault_safety_collateral_ratio_lag30          0.6249188397208192
eth_a_vault_market_collateral_ratio_lag30          0.614666336421359
psm_lifetime_turnover                              0.6049245642873874
3_m_tbill_yield                                    0.5839417913763818
effective_funds_rate                               0.5765157479244135
cumulative_revenues                                0.5715081710021795
cumulative_net_income                              0.5080156413328435
dai_total_balance                                  0.48388217558818014
dai_percent_out_dsr                                0.4705578350142762
dsr_rate             

In [374]:
# Select correlations of all features with the target variable, excluding the target variable itself
dai_target_correlations = correlations['dai_total_balance'].drop('dai_total_balance')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
dai_sorted_correlations = dai_target_correlations.sort_values(ascending=False)

for index, value in dai_sorted_correlations.items():
    print(f"{index:50} {value}")

dai_circulating                                    0.9803441280360398
dai_percent_out_dsr                                0.9759780719719016
3_m_tbill_yield                                    0.9212538850263818
effective_funds_rate                               0.9001627619326987
psm_lifetime_turnover                              0.8622720662998148
cumulative_revenues                                0.7351517263582567
sticky_cpi                                         0.7311772030422192
b_s_Real-World Assets                              0.7044493365181824
b_s_Real-World_Assets_rolling_avg                  0.7011917979014107
cumulative_net_income                              0.6439462252846612
eth_a_vault_safety_collateral_ratio_lag30          0.6331392918478008
b_s_Others_assets_rolling_avg                      0.6300646197827272
b_s_Others assets                                  0.6236625662218369
eth_a_vault_market_collateral_ratio_lag30          0.6023763633666922
eth_a_vault_safety_c

In [427]:
# Select correlations of all features with the target variable, excluding the target variable itself
sb_target_correlations = correlations['daily_surplus_buffer'].drop('daily_surplus_buffer')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
sb_sorted_correlations = sb_target_correlations.sort_values(ascending=False)

for index, value in sb_sorted_correlations.items():
    print(f"{index:50} {value}")

dai_maturity_outflow_surplus_buffer_1-year         0.9901743483451282
3.7 - Equity (Surplus Buffer)                      0.9746395333866137
where_is_dai_EOA                                   0.93990438227667
psm_lifetime_fees                                  0.865636125322519
eth_a_vault_dai_floor                              0.8470973288356538
sticky_cpi                                         0.8466611655413736
dai_maturity_outflow_1-year                        0.7876869760070996
dai_maturity_outflow_dai_only_1-year               0.781035815729949
b_s_Stablecoins_rolling_avg                        0.7539390533409592
b_s_Stablecoins                                    0.7467853841164048
b_s_Others assets                                  0.7457914770305075
b_s_Others_assets_rolling_avg                      0.7406027961038805
2.2 - Trading Assets                               0.7181724704348985
psm_balance                                        0.7141839636969732
cumulative_net_income   

In [431]:

# Select correlations of all features with the target variable, excluding the target variable itself
ni_target_correlations = correlations['1.9 - Net Income'].drop('1.9 - Net Income')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
ni_sorted_correlations = ni_target_correlations.sort_values(ascending=False)

for index, value in ni_sorted_correlations.items():
    print(f"{index:50} {value}")

Total Revenues                                     0.9124313875089496
ROA                                                0.8894602971029851
profit_margin                                      0.8894602971029851
1.1 - Lending Revenues                             0.8083327971717559
eth_a_vault_total_ann_revenues                     0.6900794385437758
eth_a_vault_annualized stability fee_lag30         0.632369301082226
eth_a_vault_annualized stability fee_90d_ma        0.6315090027654996
eth_a_vault_annualized stability fee               0.594461311734676
btc_market_Close_30d_ma                            0.5855675241300378
ROE                                                0.5826457835955418
eth_a_vault_annualized_revenues                    0.566910764875092
mkr_market_Close_7d_ma                             0.5645940375248284
eth_a_vault_daily_revenues                         0.5620243419178836
Total_Revenues_Lag_1                               0.5599858612439028
eth_a_vault_daily_reven

In [429]:
# Select correlations of all features with the target variable, excluding the target variable itself
dai_market_Volume_target_correlations = correlations['dai_market_Volume'].drop('dai_market_Volume')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
dai_market_Volume_sorted_correlations = dai_market_Volume_target_correlations.sort_values(ascending=False)

for index, value in dai_market_Volume_sorted_correlations.items():
    print(f"{index:50} {value}")

dai_market_Volume_7d_ma                            0.7694704316998318
eth_market_Volume                                  0.6064953242078771
dai_market_Volume_30d_ma                           0.6034313873288162
b_s_Crypto-Loans_rolling_avg                       0.5548466313272022
b_s_Crypto-Loans                                   0.5463016005652909
ROA_Lag_7                                          0.5246271522912113
profit_margin_Lag_7                                0.5246271522912113
eth_market_Volume_7d_ma                            0.520872484773236
eth_a_vault_collateral_usd_7d_ma                   0.507558247294655
eth_a_vault_debt_balance_7d_ma                     0.4900718803385421
profit_margin_Lag_6                                0.4892433932561513
ROA_Lag_6                                          0.4892433932561513
eth_a_vault_prev_dai_ceiling                       0.48857051141937086
eth_a_vault_collateral_usd_30d_ma                  0.4870455458711481
eth_a_vault_dai_ceili

In [428]:
# Select correlations of all features with the target variable, excluding the target variable itself
dai_treasury_target_correlations = correlations['where_is_dai_Treasury'].drop('where_is_dai_Treasury')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
dai_t_sorted_correlations = dai_treasury_target_correlations.sort_values(ascending=False)

for index, value in dai_t_sorted_correlations.items():
    print(f"{index:50} {value}")

Total_Revenues_Lag_8                               0.5519982089582511
daily_surplus_buffer                               0.538787666054152
dai_maturity_outflow_surplus_buffer_1-year         0.5053544635624468
3.7 - Equity (Surplus Buffer)                      0.4962084291367374
eth_a_vault_dai_floor                              0.4769258813378755
sticky_cpi                                         0.4713602581725595
b_s_Stablecoins_rolling_avg                        0.46373511686009977
b_s_Stablecoins                                    0.45653473238809955
psm_balance                                        0.448449977413799
where_is_dai_EOA                                   0.4416666045418965
2.2 - Trading Assets                               0.42553995011002044
1.9 - Net Income_rolling_avg_pct_chg_lag_12        0.4175115156840678
Net_Income_Lag_9                                   0.4152861613104318
dai_circulating                                    0.40430640532545753
Total_Revenues_Lag

In [426]:
# Select correlations of all features with the target variable, excluding the target variable itself
dai_p_target_correlations = correlations['dai_market_Close'].drop('dai_market_Close')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
dai_p_sorted_correlations = dai_p_target_correlations.sort_values(ascending=False)

for index, value in dai_p_sorted_correlations.items():
    print(f"{index:50} {value}")

debt_to_equity_Lag_7                               0.6120168988268051
debt_to_equity_Lag_2                               0.5857667436953803
debt_to_equity_Lag_3                               0.5789025544376855
ROE_Lag_1                                          0.5578778179979771
debt_to_equity_Lag_4                               0.5533278086345387
ROE_Lag_2                                          0.5385222248935791
b_s_Equity_rolling_avg                             0.5301078337533965
b_s_Equity                                         0.5293969791935087
debt_to_equity_Lag_6                               0.5258437003635129
debt_to_equity_Lag_1                               0.5173824695324983
eth_a_vault_liquidation_ratio                      0.5089295456879575
dai_market_Close_daily_returns                     0.5044002738393341
Total Revenues_volatility_pct_chg_lag_1            0.5033753203889272
Total Revenues_rolling_avg_pct_chg_lag_1           0.5025205109840889
debt_ratio_Lag_3    

In [375]:
# Select correlations of all features with the target variable, excluding the target variable itself
mkr_target_correlations = correlations['mkr_market_Close'].drop('mkr_market_Close')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
mkr_target_correlations = mkr_target_correlations.sort_values(ascending=False)

for index, value in mkr_target_correlations.items():
    print(f"{index:50} {value}")

mkr_market_Close_7d_ma                             0.9832239084323248
mkr_market_Close_30d_ma                            0.8996578704202103
eth_a_vault_hypothetical_dai_ceiling               0.837207589354274
eth_a_vault_annualized_revenues                    0.8365616806125646
eth_a_vault_collateral_usd                         0.8303355791576112
eth_a_vault_daily_revenues                         0.8267471989457971
eth_a_vault_usd_safety_value                       0.8229993652733495
eth_a_vault_collateral_usd_7d_ma                   0.8191820582094417
btc_market_Close_30d_ma                            0.8128904346440006
eth_a_vault_daily_revenues_7d_ma                   0.8043427714790372
btc_market_Close_7d_ma                             0.791415055963069
eth_a_vault_collateral_usd_30d_ma                  0.7825159607810909
btc_market_Close                                   0.7808343654982153
eth_a_vault_daily_revenues_30d_ma                  0.7777093293412046
eth_a_vault_dart      

In [ ]:
# Select correlations of all features with the target variable, excluding the target variable itself
mkr_target_correlations = correlations['mkr_market_Close'].drop('mkr_market_Close')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
mkr_target_correlations = mkr_target_correlations.sort_values(ascending=False)

for index, value in mkr_target_correlations.items():
    print(f"{index:50} {value}")

Next regression analysis (single, multiple) for target variable and strong correlating features

In your model, which aims to simulate collateral balance for a vault and use Reinforcement Learning (RL) for Maker Protocol financial management with the Dai Savings Rate (DSR) and Stability Fee as part of the action space, the variables can be categorized as follows:

Independent Variables
These are the input features that your model will use to make predictions or decisions. Based on the extensive list of features you've provided and the objectives of your model, independent variables may include:

Market data (e.g., eth_market_Close, btc_market_Close, dai_market_Close)
Operational metrics from the vault (e.g., eth_a_vault_market_price, eth_a_vault_collateral_usd)
Financial metrics (e.g., cumulative_revenues, cumulative_expenses, profit_margin)
DAI metrics (e.g., dai_circulating, dai_percent_in_dsr, dsr_balance)
Macro-economic indicators (e.g., effective_funds_rate, 3_m_tbill_yield, sticky_cpi)
These variables are considered independent because they are assumed not to be influenced by the model's outputs but rather to affect them.

Dependent Variables
These are the outcomes or targets your model aims to predict or optimize. In the context of your model's objectives, the dependent variables could be:

For the simulation segment:

eth_a_vault_cumulative_collateral: This represents the total collateral balance in an ETH vault, which you aim to simulate as a response to changes in policy variables.
For the RL segment:

dsr_rate: The Dai Savings Rate, which is a policy tool MakerDAO uses to influence various aspects of the ecosystem.
stability_fee: The fee applied to vaults, which acts as another policy lever to maintain the peg of DAI to its underlying asset and control supply dynamics.
The choice of dependent and independent variables will depend on the specific objectives of each segment of your model (simulation and RL). The model aims to learn how changes in independent variables (market conditions, operational metrics, financial metrics, macroeconomic indicators, etc.) influence the dependent variables (collateral balance for the simulation part and the optimal DSR and Stability Fee rates for the RL part). This setup allows the model to simulate the impact of policy decisions on the vault's collateral balance and to optimize these decisions to achieve desired outcomes in the MakerDAO ecosystem.

When the Debt Ceiling parameter for a vault type is set lower than the current amount of DAI outstanding (i.e., the total amount of DAI minted from that vault type), it does not directly affect the existing DAI or vaults. Here's what happens:

No Immediate Effect on Existing DAI: Lowering the Debt Ceiling below the current amount of DAI outstanding does not trigger automatic liquidations or directly impact the existing DAI in circulation. The existing DAI remains valid, and holders of DAI are not directly affected by this change in the Debt Ceiling.

Prevention of New DAI Minting: The primary consequence of lowering the Debt Ceiling below the current DAI outstanding is the prevention of new DAI being minted from that specific vault type. This means that users cannot increase their debt (i.e., mint more DAI) from vaults of that type until the total debt is reduced below the new Debt Ceiling level or until the Debt Ceiling is raised again.

Encourages DAI Repayment: By setting a lower Debt Ceiling, the Maker Governance effectively limits the potential for further exposure to the collateral type associated with that vault. This may encourage users to repay some of their DAI debt to free up space under the Debt Ceiling, allowing for new minting activities, or to shift their operations to other vault types with available capacity.

Governance and Risk Management Tool: Lowering the Debt Ceiling can be a proactive measure by Maker Governance to manage risk, especially if there are concerns about overexposure to a particular asset type or if there's a desire to mitigate potential impacts of the 'OSM Timing Attack.' It's a governance tool used to control the risk profile of the MakerDAO system and ensure the stability and security of the DAI peg to the USD.

In summary, setting a Debt Ceiling lower than the current amount of DAI minted does not affect the existing DAI but prevents further minting of DAI from the affected vault type. This measure serves as a risk management and governance tool, allowing Maker Governance to control exposure to specific assets and maintain the system's stability.